# Computation of resolution performance



## Setup

In [1]:
import sys
sys.path.insert(0, '..')
sys.path.insert(1, 'python/')

import ROOT
import root_numpy as rnp
import python.l1THistos as histos
import math
import uuid
import pandas as pd

from drawingTools import *

ROOT.enableJSVis()
#ROOT.enableJSVis()

#from drawingTools import *

normalized_histos = list()
    

Welcome to JupyROOT 6.14/02


In [2]:
# %load python/drawingTools
# %load python/drawingTools.py
# %load python/drawingTools.py
# %load drawingTools.py
import ROOT
import math
import uuid
import pandas as pd


# some useful globals, mainly to deal with ROOT idiosyncrasies
c_idx = 0
p_idx = 0
colors = [1, 632-4, 416+1, 600-3, 616+1, 432-3]
colors.extend(range(1, 12))


marker_styles = [8, 21, 22, 23, 33, 33, 33, 33, 33, 33, 33, 33]
stuff = []
f_idx = 0

ROOT.gStyle.SetOptTitle(False)
ROOT.gStyle.SetPadBottomMargin(0.13)
ROOT.gStyle.SetPadLeftMargin(0.13)
ROOT.gStyle.SetPadRightMargin(0.13)
ROOT.gStyle.SetOptStat(False)

# ROOT.gStyle.SetCanvasBorderMode(0)
# ROOT.gStyle.SetCanvasColor(0)


# def DrawPrelimLabel(canvas):
#     canvas.cd()
#     tex = ROOT.TLatex()
#     global stuff
#     stuff.append(tex)
#     tex.SetTextSize(0.03)
#     tex.DrawLatexNDC(0.13,0.91,"#scale[1.5]{CMS} #scale[1.]{Phase-2 Simulation}")
#     tex.DrawLatexNDC(0.49,0.91,"14TeV, 7.5#times10^{34}cm^{-2}s^{-1}, 200 PU")

#     tex.Draw("same");
#     return

def SaveCanvas(canvas, name):
    canvas.cd()
    canvas.SaveAs(name+'.pdf')


# void SaveCanvas(TCanvas* c, TString PlotName = "myPlotName")
# {
#   c->cd();
#   c->SaveAs(PlotName + ".pdf");
#   c->SaveAs(PlotName + ".root");

#   return;
# }



def getText(text, ndc_x, ndc_y):
    global stuff
    rtext = ROOT.TLatex(ndc_x, ndc_y, text)
    stuff.append(rtext)
    rtext.SetNDC(True)
    # rtext.SetTextFont(40)
    rtext.SetTextSize(0.03)
    return rtext


def getLegend(x1=0.7, y1=0.71, x2=0.95, y2=0.85):
    global stuff
    legend = ROOT.TLegend(x1, y1, x2, y2)
    stuff.append(legend)
    legend.SetFillColor(0)
    legend.SetFillStyle(0)
    legend.SetBorderSize(0)
    legend.SetTextSize(0.03)
    return legend


def newCanvas(name=None, title=None, height=600, width=800, xdiv=0, ydiv=0, form=4):
    global c_idx
    if name is None:
        name = 'c_{}'.format(uuid.uuid4().hex[:6])
        c_idx += 1
    if title is None:
        title = name
    # print name, title, width, height
    canvas = ROOT.TCanvas(name, title, width, height)
    if(xdiv*ydiv != 0):
        canvas.Divide(xdiv, ydiv)
    global stuff
    stuff.append(canvas)
    return canvas


def drawAndProfileX(plot2d, miny=None, maxy=None, do_profile=True, options='', text=None):
    global p_idx
    if miny and maxy:
        plot2d.GetYaxis().SetRangeUser(miny, maxy)
    c = newCanvas()
    c.SetGrid(1, 1)
    c.cd()
    plot2d.Draw(options)
    ROOT.gPad.SetGrid(1, 1)
    ROOT.gStyle.SetGridColor(15)

    if do_profile:
        profname = plot2d.GetName()+'_prof_'+str(p_idx)
        p_idx += 1
        firstbin = 1
        lastbin = -1
        prof = plot2d.ProfileX(profname, firstbin, lastbin, 's')
        prof.SetMarkerColor(2)
        prof.SetLineColor(2)
        prof.Draw('same')

    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw('same')


    c.Draw()

class DrawConfig(object):
    def __init__(self):
        self.do_stats = False
        self.marker_size = 0.5
        self.marker_styles = [8, 21, 22, 23, 33]
        self.colors = [1, 632-4, 416+1, 600-3, 616+1, 432-3]
#         self.canvas_sizes = (800, 600)
        self.canvas_sizes = (600, 600)
        # SetMargin (Float_t left, Float_t right, Float_t bottom, Float_t top)
#         self.canvas_margins = (0.13, 0.3, 0.13, 0.1)
        self.canvas_margins = (0.13, 0.13, 0.13, 0.1)
        self.canvas_margins_div = (0.13, 0.13, 0.13, 0.1)
        self.legend_position = (0.6, 0.7)
        self.legend_size = (0.26, 0.1)
        self.additional_text = []
        self.additional_text_size = 0.03
        return

tdr_config = DrawConfig()
tdr_config.additional_text.append((0.13,0.91,"#scale[1.5]{CMS} #scale[1.]{Phase-2 Simulation}"))
tdr_config.additional_text.append((0.69,0.91,"14TeV, 200 PU"))

rleg_config = DrawConfig()
rleg_config.canvas_sizes = (800, 600)
rleg_config.canvas_margins = (0.13, 0.3, 0.13, 0.1)
rleg_config.legend_position = (0.7, 0.71)
rleg_config.legend_size = (0.25, 0.14)


class DrawMachine(object):
    def __init__(self, config):
        global stuff
        self.config = config
        self.histos = []
        stuff.append(self.histos)
        self.labels = []
        self.overlay = True
        self.canvas = None
        self.legend = None
        return

    def addHistos(self, histograms, labels):
        for hidx,hist in enumerate(histograms):
            histo_class = hist.ClassName()
            if 'TH2' in histo_class or 'TH3' in histo_class:
                self.overlay = False

            # clone the histo
            d_hist = hist.Clone(uuid.uuid4().hex[:6])
#             if 'TGraph' in histo_class:
#                 d_hist.SetTitle(';'+';'.join(hist.GetTitle().split(';')[1:]))
#             else:
            d_hist.SetTitle(';{};{}'.format(hist.GetXaxis().GetTitle(), hist.GetYaxis().GetTitle()))
            # drop the title
            #             d_hist.SetTitle("")
            self.histos.append(d_hist)
            self.labels.append(labels[hidx])
        return


    def drawAdditionalText(self):
        tex = ROOT.TLatex()
        global stuff
        stuff.append(tex)
        tex.SetTextSize(self.config.additional_text_size)
        for txt in self.config.additional_text:
            tex.DrawLatexNDC(txt[0], txt[1], txt[2])
        tex.Draw("same");


    def formatHistos(self):
        for hidx,hist in enumerate(self.histos):
            histo_class = hist.ClassName()
            hist.UseCurrentStyle()

            if 'TGraph' in histo_class:
                hist.SetMarkerSize(self.config.marker_size)
                hist.SetMarkerStyle(self.config.marker_styles[hidx])
                if self.overlay:
                    hist.SetMarkerColor(colors[hidx])
                    hist.SetLineColor(self.config.colors[hidx])
            elif 'TF1' in histo_class:
                hist.SetLineColor(self.config.colors[hidx])
            else:
                hist.SetStats(self.config.do_stats)
                if self.overlay:
                    hist.SetLineColor(self.config.colors[hidx])
        return

    def createCanvas(self, do_ratio=False):
        if self.canvas is not None:
            return

        xdiv = 0
        ydiv = 0

        c_width, c_height = self.config.canvas_sizes

        if not self.overlay:
            xdiv = 2
            ydiv = math.ceil(float(len(self.histos))/2)
            c_width = 1000.
            c_height = 500*ydiv
        else:
            if do_ratio:
                c_height = c_height+200

        self.canvas = newCanvas(name=None,
                                title=None,
                                height=int(c_height),
                                width=int(c_width),
                                xdiv=int(xdiv),
                                ydiv=int(ydiv))
        if self.overlay:
            self.canvas.SetMargin(self.config.canvas_margins[0],
                                  self.config.canvas_margins[1],
                                  self.config.canvas_margins[2],
                                  self.config.canvas_margins[3])
        else:
            self.canvas.SetMargin(self.config.canvas_margins_div[0],
                                  self.config.canvas_margins_div[1],
                                  self.config.canvas_margins_div[2],
                                  self.config.canvas_margins_div[3])
        return

    def createLegend(self):
        if self.legend is not None:
            return
        self.legend = getLegend(self.config.legend_position[0],
                                self.config.legend_position[1],
                                self.config.legend_position[0]+self.config.legend_size[0],
                                self.config.legend_position[1]+self.config.legend_size[1])
        for hidx,hist in enumerate(self.histos):
            histo_class = hist.ClassName()
            if 'TGraph' not in histo_class:
                self.legend.AddEntry(hist, self.labels[hidx], 'l')
            else:
                self.legend.AddEntry(hist, self.labels[hidx], 'P')

        return


    def draw(self,
             text,
             options='',
             norm=False,
             y_log=False,
             x_log=False,
             y_min=None,
             y_max=None,
             x_min=None,
             x_max=None,
             y_axis_label=None,
             x_axis_label=None,
             v_lines=None,
             h_lines=None,
             do_profile=False):

        global p_idx
        global stuff

        self.formatHistos()
        self.createCanvas()
        if self.overlay:
            self.createLegend()

        self.canvas.cd()

        drawn_histos = []
        for hidx, hist in enumerate(self.histos):
            histo_class = hist.ClassName()

            opt = options
            if 'TGraph' in histo_class:
                opt = 'P'+options
            if hidx:
                if self.overlay:
                    opt = 'same,'+opt
            else:
                if 'TGraph' in histo_class:
                    opt = opt+'A'

            if not self.overlay:
                self.canvas.cd(hidx+1)

            d_hist = hist
            if norm:
                d_hist = hist.DrawNormalized(opt, 1.)
            else:
                d_hist.Draw(opt)

            if not self.overlay:
                if text:
                    newtext = '{}: {}'.format(self.labels[hidx], text)
                    rtext = getText(newtext, 0.15, 0.85)
                    rtext.Draw('same')
                    self.drawAdditionalText()

            if do_profile:
                profname = d_hist.GetName()+'_prof_'+str(p_idx)
                p_idx += 1
                prof = d_hist.ProfileX(d_hist.GetName()+'_prof_'+str(p_idx),
                                       1, -1, 's')
                prof.SetMarkerColor(2)
                prof.SetLineColor(2)
                prof.Draw('same')


            drawn_histos.append(d_hist)

            # we now set the axis properties
            y_min_value = y_min
            y_max_value = y_max

            if y_min is None:
                y_min_value = min([hist.GetBinContent(hist.GetMinimumBin()) for hist in drawn_histos if 'TGraph' not in hist.ClassName() and 'TF1' not in hist.ClassName()] +
                                  [min(hist.GetY()) for hist in drawn_histos if 'TGraph' in hist.ClassName() and 'TF1' not in hist.ClassName()])
            if y_max is None:
                y_max_value = max([hist.GetBinContent(hist.GetMaximumBin()) for hist in drawn_histos if 'TGraph' not in hist.ClassName() and 'TF1' not in hist.ClassName()] +
                                  [max(hist.GetY()) for hist in drawn_histos if 'TGraph' in hist.ClassName() and 'TF1' not in hist.ClassName()])*1.2

            for hist in drawn_histos:
                hist.GetXaxis().SetTitleOffset(1.4)
                hist.GetYaxis().SetRangeUser(y_min_value, y_max_value)
                if y_axis_label:
                    hist.GetYaxis().SetTitle(y_axis_label)
                if x_axis_label:
                    hist.GetXaxis().SetTitle(x_axis_label)
                if x_min is not None and x_max is not None:
                    if 'TGraph' not in hist.ClassName():
                        hist.GetXaxis().SetRangeUser(x_min, x_max)
                    else:
                        hist.GetXaxis().SetLimits(x_min, x_max)


        if self.legend is not None and len(self.histos) > 1:
            self.legend.Draw("same")

        if self.overlay:
            if text:
                rtext = getText(text, 0.15, 0.85)
                rtext.Draw("same")
            self.drawAdditionalText()

        pad_range = range(0, 1)
        if not self.overlay:
            pad_range = range(1, len(self.histos)+1)

        self.canvas.Update()
        for pad_id in pad_range:
            self.canvas.cd(pad_id)
            if v_lines:
                for v_line_x in v_lines:
                    aline = ROOT.TLine(v_line_x, ROOT.gPad.GetUymin(), v_line_x,  ROOT.gPad.GetUymax())
                    aline.SetLineStyle(2)
                    aline.Draw("same")
                    stuff.append(aline)
            if h_lines:
                for h_line_y in h_lines:
                    aline = ROOT.TLine(ROOT.gPad.GetUxmin(), h_line_y, ROOT.gPad.GetUxmax(),  h_line_y)
                    aline.SetLineStyle(2)
                    aline.Draw("same")
                    stuff.append(aline)
            if y_log:
                ROOT.gPad.SetLogy()
            if x_log:
                ROOT.gPad.SetLogx()

            ROOT.gPad.Update()

        self.canvas.Draw()
        return

    def write(self, name, ext='pdf'):
        self.canvas.SaveAs('{}.{}'.format(name, ext))
        return


def draw(histograms,
         labels,
         options='',
         norm=False,
         logy=False,
         min_y=None,
         max_y=None,
         min_x=None,
         max_x=None,
         text=None,
         y_axis_label=None,
         x_axis_label=None,
         v_lines=None,
         h_lines=None,
         do_stats=False,
         do_profile=False,
         do_ratio=False,
         do_write=False,
         write_name=None,
         legend_position=None):
    ROOT.gStyle.SetOptStat(False)

    config = rleg_config
    if do_write:
        config = tdr_config
    if do_stats:
        config.do_stats = do_stats
    if legend_position is not None:
        config.legend_position = (legend_position[0],
                                  legend_position[1])
        config.legend_size = (legend_position[2]-legend_position[0],
                              legend_position[3]-legend_position[1])

    dm = DrawMachine(config)
    dm.addHistos(histograms, labels)
    dm.draw(text=text,
            options=options,
            norm=norm,
            y_log=logy,
            y_min=min_y,
            y_max=max_y,
            x_min=min_x,
            x_max=max_x,
            y_axis_label=y_axis_label,
            x_axis_label=x_axis_label,
            v_lines=v_lines,
            h_lines=h_lines,
            do_profile=do_profile,
            # do_ratio=False,
           )
    if do_write:
        dm.write(name=write_name)
    return dm


files = {}
file_keys = {}


class RootFile:
    def __init__(self, file_name):
        global file
        self.file_name = file_name
        if self.file_name not in files.keys():
            print 'get file: {}'.format(self.file_name)
            files[self.file_name] = ROOT.TFile(self.file_name)
        self._file = files[self.file_name]
        self._file_keys = None

    def cd(self):
        self._file.cd()

    def GetListOfKeys(self):
        global file_keys
        if self.file_name not in file_keys.keys():
            print 'get list'
            file_keys[self.file_name] = self._file.GetListOfKeys()
        self._file_keys = file_keys[self.file_name]
        return self._file_keys


class Sample():
    def __init__(self, name, label, version=None, type=None):
        self.name = name
        self.label = label
        if version:
            version = '_'+version
        else:
            version = ''
        self.histo_filename = '../plots1/histos_{}{}.root'.format(self.name, version)
        self.histo_file = None
        self.type = type
        self.oldStyle = False

    def __repr__(self):
        return '<{} {}, {}>'.format(self.__class__.__name__, self.histo_filename, self.type)

    def open_file(self):
        if self.histo_file is None:
            self.histo_file = ROOT.TFile(self.histo_filename, 'r')

    def build_file_primitive_index_oldStyle(self):
        # FIXME: this is really hugly
        composite_classes = {('GenParticleHistos', 'h_effNum_'): 'HistoSetEff',
                             ('GenParticleHistos', 'h_effDen_'): 'HistoSetEff',
                             ('TCHistos', 'h_tc_'): 'HistoSetClusters',
                             ('ClusterHistos', 'h_cl2d_'): 'HistoSetClusters',
                             ('Cluster3DHistos', 'h_cl3d_'): 'HistoSetClusters'}
        self.open_file()
        self.histo_file.cd()
        data_list = []
        classtype = 'CalibrationHistos'
        print '--- {}'.format(classtype)
        print '# of plots: {}'.format(len(self.histo_file.GetListOfKeys()))
        # same primitives (tp, tp_sel, gen_sel) applies to several entries
        key_set = set()
        for histo in self.histo_file.GetListOfKeys():
            # print histo.GetName()
            name_parts = histo.GetName().split('_')
            cltype, tp, tp_sel, gen_sel = None, None, None, None
            if len(name_parts) == 4:
                cltype, tp, tp_sel, gen_sel = classtype, name_parts[0], name_parts[1], name_parts[2]
            else:
                # this is a histo in a HistoSet class.. we need to handle this differently
                composite_class = composite_classes[(classtype, '{}_{}_'.format(name_parts[0], name_parts[1]))]
                cltype, tp, tp_sel, gen_sel = composite_class, name_parts[2], name_parts[3], name_parts[4]
            key_set.add((cltype, tp, tp_sel, gen_sel))
        print '# of primitives: {}'.format(len(key_set))
        for cltype, tp, tp_sel, gen_sel in key_set:
            data_list.append({'classtype': cltype,
                              'tp': tp,
                              'tp_sel': tp_sel,
                              'gen_sel': gen_sel})

        return pd.DataFrame(data_list)

    def build_file_primitive_index(self):
        if self.oldStyle:
            return self.build_file_primitive_index_oldStyle()
        # FIXME: this is really hugly
        composite_classes = {('GenParticleHistos', 'h_effNum_'): 'HistoSetEff',
                             ('GenParticleHistos', 'h_effDen_'): 'HistoSetEff',
                             ('TCHistos', 'h_tc_'): 'HistoSetClusters',
                             ('ClusterHistos', 'h_cl2d_'): 'HistoSetClusters',
                             ('Cluster3DHistos', 'h_cl3d_'): 'HistoSetClusters',
                             ('ResoHistos', 'h_reso_'): 'ResoHistos'}

        self.open_file()
        self.histo_file.cd()
        data_list = []
        for key in self.histo_file.GetListOfKeys():
            # first level is classtype
            classtype = key.GetName()
            print '--- {}'.format(classtype)
            file_dir = self.histo_file.GetDirectory(key.GetName())
            print '# of plots: {}'.format(len(file_dir.GetListOfKeys()))
            # same primitives (tp, tp_sel, gen_sel) applies to several entries
            key_set = set()
            for histo in file_dir.GetListOfKeys():
                # print histo.GetName()
                name_parts = histo.GetName().split('_')
                cltype, tp, tp_sel, gen_sel = None, None, None, None
                if len(name_parts) == 3:
                    cltype, tp, tp_sel, gen_sel = classtype, name_parts[0], name_parts[1], None
                elif len(name_parts) == 4:
                    cltype, tp, tp_sel, gen_sel = classtype, name_parts[0], name_parts[1], name_parts[2]
                else:
                    # this is a histo in a HistoSet class.. we need to handle this differently
                    composite_class = composite_classes[(classtype, '{}_{}_'.format(name_parts[0], name_parts[1]))]
                    cltype, tp, tp_sel, gen_sel = composite_class, name_parts[2], name_parts[3], name_parts[4]
                key_set.add((cltype, tp, tp_sel, gen_sel))
            print '# of primitives: {}'.format(len(key_set))
            for cltype, tp, tp_sel, gen_sel in key_set:
                data_list.append({'classtype': cltype,
                                  'tp': tp,
                                  'tp_sel': tp_sel,
                                  'gen_sel': gen_sel})

        return pd.DataFrame(data_list)

    def print_file_primitive_index(self):
        primitive_index = self.build_file_primitive_index()
        classtypes = primitive_index.classtype.unique()
        for classtype in classtypes:
            print '- HistoClass: {}'.format(classtype)
            tps = primitive_index[primitive_index.classtype == classtype].tp.unique()
            for tp in tps:
                print '  - TP: {}'.format(tp)
                tp_sels = primitive_index[(primitive_index.classtype == classtype) &
                                          (primitive_index.tp == tp)].tp_sel.unique()
                for tp_sel in tp_sels:
                    gen_sels = primitive_index[(primitive_index.classtype == classtype) &
                                               (primitive_index.tp == tp) &
                                               (primitive_index.tp_sel == tp_sel)].gen_sel.unique()
                    print '    - TP SEL: {} -> GEN SEL: {}'.format(tp_sel, gen_sels)


class HProxy:
    def __init__(self, classtype, tp, tp_sel, gen_sel, root_file):
        self.classtype = classtype
        self.tp = tp
        self.tp_sel = tp_sel
        self.gen_sel = gen_sel
        self.root_file = root_file
        self.instance = None

    def get(self, debug=False):
        if self.instance is None:
            name = '{}_{}_{}'.format(self.tp, self.tp_sel, self.gen_sel)
            if self.gen_sel is None:
                name = '{}_{}'.format(self.tp, self.tp_sel)
            if debug:
                print '-- HProxy:Get: {}'.format(name)
            self.instance = self.classtype(name, self.root_file, debug)
        return self.instance


class HPlot:
    def __init__(self, samples, labels_dict):
        self.samples_ = samples

        # populate the label dict
        self.labels_dict = labels_dict
        for sample in samples:
            self.labels_dict[sample.type] = sample.type
        self.labels_dict.update({'PU0': 'PU0', 'PU200': 'PU200'})

        self.data = pd.DataFrame(columns=['sample', 'pu', 'tp', 'tp_sel', 'gen_sel', 'classtype', 'histo'])

    def create_histo_proxies(self, classtype):
        for sample in self.samples_:
            histo_primtive_index = sample.build_file_primitive_index()
            class_primitive_index = histo_primtive_index[histo_primtive_index.classtype == str(classtype).split('.')[-1]]
            if class_primitive_index.empty:
                print '*** ERROR: No entry for class: {} in the primtive index!'.format(classtype)
                return

            for index, row in class_primitive_index.iterrows():
                self.data = self.data.append({'sample': sample.type,
                                              'pu': sample.label,
                                              'tp': row.tp,
                                              'tp_sel': row.tp_sel,
                                              'gen_sel': row.gen_sel,
                                              'classtype': classtype,
                                              'histo': HProxy(classtype,
                                                              row.tp,
                                                              row.tp_sel,
                                                              row.gen_sel,
                                                              sample.histo_file)},
                                             ignore_index=True)
            self.data.drop_duplicates(
                subset=['sample', 'pu', 'tp', 'tp_sel', 'gen_sel', 'classtype'],
                inplace=True,
                keep='last')

    def get_histo(self,
                  classtype,
                  sample=None,
                  pu=None,
                  tp=None,
                  tp_sel=None,
                  gen_sel=None,
                  debug=False):
        histo = None
        labels = []
        text = ''

        query = '(pu == @pu) & (tp == @tp) & (tp_sel == @tp_sel) & (classtype == @classtype)'
        if gen_sel is not None:
            query += ' & (gen_sel == @gen_sel)'
        else:
            query += ' & (gen_sel.isnull())'
        if sample is not None:
            query += '& (sample == @sample)'

        histo_df = self.data.query(query)

        if histo_df.empty:
            print 'No match found for: pu: {}, tp: {}, tp_sel: {}, gen_sel: {}, classtype: {}'.format(pu, tp, tp_sel, gen_sel, classtype)
            return None, None, None
        if debug:
            print histo_df

        field_counts = histo_df.apply(lambda x: len(x.unique()))
        label_fields = []
        text_fields = []
        # print field_counts
        for field in field_counts.iteritems():
            if(field[1] > 1 and field[0] != 'histo'):
                label_fields.append(field[0])
            if(field[1] == 1 and field[0] != 'histo' and field[0] != 'classtype' and field[0] != 'sample'):
                if(gen_sel is None and field[0] == 'gen_sel'):
                    continue
                text_fields.append(field[0])
#         print 'label fields: {}'.format(label_fields)
#         print 'text fields: {}'.format(text_fields)

        for item in histo_df[label_fields].iterrows():
            labels.append(', '.join([self.labels_dict[tx] for tx in item[1].values if self.labels_dict[tx] != '']))

        # print labels
        text = ', '.join([self.labels_dict[fl] for fl in histo_df[text_fields].iloc[0].values if self.labels_dict[fl] != ''])
        histo = [his.get(debug) for his in histo_df['histo'].values]
        return histo, labels, text


In [3]:
# %load samples.py
import ROOT
import pandas as pd
import python.selections as selections

version_V9 = 'v140D'



# -------------------------------------------------------------------------

samples_ele_V9 = [
    Sample('ele_flat2to100_PU0_reso', 'PU0', version_V9, 'ele-V9'),
    Sample('ele_flat2to100_PU200_reso', 'PU200', version_V9, 'ele-V9')
    ]

samples_photons_V9 = [
    Sample('photon_flat8to150_PU0_reso', 'PU0', version_V9, 'photons-V9'),
    Sample('photon_flat8to150_PU200_reso', 'PU200', version_V9, 'photons-V9')
    ]



gen_part_selections: 10


In [4]:
# %load python/settings.py

# === samples =====================================================
import pprint
import python.plotters_config as plotters



samples = []

# samples += samples_nugunrates
# samples += samples_nugunrates_V8
samples += samples_ele_V9
# samples += samples_photons_V9

for smp in samples:
    smp.open_file()

sample = 'ele-V9'

# === TP ==========================================================
tps = ['EG', 'HMvDR', 'HMvDRshapeDr','HMvDRshapeDrCalib','HMvDRshapeDrCalibNew', 'HMvDREmInt']


# === Load the Histo Primitives ====================================
histo_primitives = samples[0].build_file_primitive_index()


# print histo_primitives.data.unique()
# === TP selections ================================================
tp_select = {}

for tp in tps:
    tp_select[tp] = histo_primitives[histo_primitives.tp == tp].tp_sel.unique().tolist()

# ==== GEN selections ===============================================
gen_select ={}
for tp in tps:
    gen_select[tp] = histo_primitives[histo_primitives.tp == tp].gen_sel.unique().tolist()


import pprint
pp = pprint.PrettyPrinter(indent=4)
print '--- TPs: '
pp.pprint(tps)
print '--- TP selections:'
pp.pprint(tp_select)
print '--- GEN selections:'
pp.pprint(gen_select)


--- ResoTuples
# of plots: 160
# of primitives: 160
--- TPs: 
[   'EG',
    'HMvDR',
    'HMvDRshapeDr',
    'HMvDRshapeDrCalib',
    'HMvDRshapeDrCalibNew',
    'HMvDREmInt']
--- TP selections:
{   'EG': ['EGq3', 'EGq4', 'EGq5', 'EGq2'],
    'HMvDR': ['Em', 'all'],
    'HMvDREmInt': ['all', 'Em'],
    'HMvDRshapeDr': ['Em', 'all'],
    'HMvDRshapeDrCalib': ['Em', 'all'],
    'HMvDRshapeDrCalibNew': ['Em', 'all']}
--- GEN selections:
{   'EG': [   'GEN',
              'GENPt15',
              'GENPt35',
              'GENEtaBC',
              'GENEtaB',
              'GENEtaBCD',
              'GENEtaD',
              'GENEtaC',
              'GENPt30',
              'GENPt40'],
    'HMvDR': [   'GENEtaD',
                 'GENEtaB',
                 'GENEtaCPt35',
                 'GENEtaBCD',
                 'GENEtaBC',
                 'GENEtaCPt40',
                 'GENEtaC',
                 'GENEtaCPt30',
                 'GEN',
                 'GENEtaCPt15'],
    'HMvDREmInt'

In [5]:
print histo_primitives.tp.unique()

['HMvDRshapeDrCalib' 'HMvDRshapeDrCalibNew' 'HMvDRshapeDr' 'HMvDRCalib'
 'HMvDR' 'HMvDREmInt' 'EG']


In [6]:
import python.collections as collections
import python.selections as selections

labels_dict = {}

evm = collections.EventManager()
labels_dict.update(evm.get_labels())
selm = selections.SelectionManager()
labels_dict.update(selm.get_labels())



In [7]:
for tp in tps:
    print 'TP: {}'.format(tp)
    print histo_primitives[histo_primitives.tp == tp].tp_sel.unique()
    print histo_primitives[histo_primitives.tp == tp].gen_sel.unique()

TP: EG
['EGq3' 'EGq4' 'EGq5' 'EGq2']
['GEN' 'GENPt15' 'GENPt35' 'GENEtaBC' 'GENEtaB' 'GENEtaBCD' 'GENEtaD'
 'GENEtaC' 'GENPt30' 'GENPt40']
TP: HMvDR
['Em' 'all']
['GENEtaD' 'GENEtaB' 'GENEtaCPt35' 'GENEtaBCD' 'GENEtaBC' 'GENEtaCPt40'
 'GENEtaC' 'GENEtaCPt30' 'GEN' 'GENEtaCPt15']
TP: HMvDRshapeDr
['Em' 'all']
['GENEtaD' 'GEN' 'GENEtaCPt40' 'GENEtaCPt15' 'GENEtaCPt35' 'GENEtaB'
 'GENEtaBCD' 'GENEtaC' 'GENEtaBC' 'GENEtaCPt30']
TP: HMvDRshapeDrCalib
['Em' 'all']
['GENEtaD' 'GEN' 'GENEtaCPt15' 'GENEtaCPt40' 'GENEtaBC' 'GENEtaBCD'
 'GENEtaCPt35' 'GENEtaB' 'GENEtaCPt30' 'GENEtaC']
TP: HMvDRshapeDrCalibNew
['Em' 'all']
['GENEtaBCD' 'GENEtaB' 'GENEtaBC' 'GEN' 'GENEtaCPt35' 'GENEtaCPt30'
 'GENEtaCPt15' 'GENEtaC' 'GENEtaD' 'GENEtaCPt40']
TP: HMvDREmInt
['all' 'Em']
['GENEtaCPt15' 'GENEtaBC' 'GEN' 'GENEtaCPt40' 'GENEtaCPt35' 'GENEtaB'
 'GENEtaBCD' 'GENEtaD' 'GENEtaC' 'GENEtaCPt30']


In [8]:
# tp_selections = [('EG', 'EGq5', 'GENEtaBC'),
#                  ('EG', 'EGq4', 'GENEtaBC'), 
#                  ('EG', 'EGq3', 'GENEtaBC'), 
#                  ('EG', 'EGq5', 'GENEtaDE'),
#                  ('EG', 'EGq5', 'GENEtaD'), 
#                  ('EGBRL', 'LooseTkID', 'GENEtaF'), 
#                  ('EG', 'EGq5', 'GEN'), 
#                  ('TkEleEL', 'EGq5', 'GENEtaBC'),
#                  ('TkEleELBRL', 'all', 'GENEtaF'), ]
tp_selections = [('EG', 'EGq5', 'GENEtaBC'),
                 ('EG', 'EGq4', 'GENEtaBC'), 
                 ('HMvDR', 'Em', 'GENEtaBC'), 
                 ('HMvDRshapeDrCalib', 'Em', 'GENEtaBC'),
                 ('HMvDRshapeDrCalibNew', 'Em', 'GENEtaBC'),
                 ('HMvDREmInt', 'Em', 'GENEtaBC'), ]




### read the calib trees

In [9]:

import array
import numpy as np 

tree_data = pd.DataFrame()

for smp in samples:
    print ""
    print '------ SAMPLE: {}, PU: {}'.format(smp.type, smp.label)
    for tp, tp_sel, gen_sel in tp_selections:
        print 'TP: {} sel: {}, gen_sel: {}'.format(tp, tp_sel, gen_sel)
    
        if 'TkEle' in tp and 'photon' in smp.type:
            continue
        
        
        tree_name = 'ResoTuples/{}_{}_{}_reso'.format(tp, tp_sel, gen_sel)

        # get the tree
        smp.histo_file.cd('ResoTuples')
        tree = smp.histo_file.Get(tree_name)
        print tree
        smp.histo_file.cd('../')
        # read the branches
        df = pd.DataFrame()

#                 cluster_pt = np.array([list(aset)[0] for aset in rnp.tree2array(tree, branches=['pt'], cache_size=400000000)])
#                 cluster_eta = np.array([list(aset)[0] for aset in rnp.tree2array(tree, branches=['eta'], cache_size=400000000)])
#                 branches = ['e1', 'e3','e5','e7','e9','e11','e13','e15','e17','e19','e21','e23','e25','e27']

        gen_energy = np.array(rnp.tree2array(tree, branches='e_gen', cache_size=400000000))
        cl_energy = np.array(rnp.tree2array(tree, branches='e', cache_size=400000000))
        gen_pt = np.array(rnp.tree2array(tree, branches='pt_gen', cache_size=400000000))
        cl_pt = np.array(rnp.tree2array(tree, branches='pt', cache_size=400000000))
        gen_eta = np.array(rnp.tree2array(tree, branches='eta_gen', cache_size=400000000))
        cl_eta = np.array(rnp.tree2array(tree, branches='eta', cache_size=400000000))

        # fill the array

#                 df['cl_pt'] = cluster_pt
#                 df['cl_layer_energies'] = cluster_layer_energies.tolist()
#                 df['cl_eta'] = cluster_eta
#                 df['cl_abseta'] = np.abs(df.cl_eta)
        df['gen_energy'] = gen_energy
        df['gen_eta'] = gen_eta
        df['cl_eta'] = cl_eta
        df['gen_pt'] = gen_pt
        df['cl_energy'] = cl_energy
        df['cl_pt'] = cl_pt

#         if tp != 'HMvDREmInt':
#             df['cl_energy'] = cl_energy
#             df['cl_pt'] = cl_pt
#         else:
#             print "apply"
#             eta_corr = (-24.9577, 52.9941)
#             df['cl_energy'] = cl_energy + (eta_corr[1]+np.abs(cl_eta)*eta_corr[0]) - (eta_corr[0]+np.abs(cl_eta)*eta_corr[1])
#             df['cl_pt'] = df.cl_energy/np.cosh(df.cl_eta)
            
        df.loc[:, 'smp'] = smp.type
        df.loc[:, 'pu'] = smp.label
        df.loc[:, 'tp'] = tp
        df.loc[:, 'gen_sel'] = gen_sel
        df.loc[:, 'tp_sel'] = tp_sel

        tree_data = tree_data.append(df, ignore_index=True)



------ SAMPLE: ele-V9, PU: PU0
TP: EG sel: EGq5, gen_sel: GENEtaBC
<ROOT.TNtuple object ("EG_EGq5_GENEtaBC_reso") at 0x7fe3757d2100>
TP: EG sel: EGq4, gen_sel: GENEtaBC
<ROOT.TNtuple object ("EG_EGq4_GENEtaBC_reso") at 0x7fe366dd4e40>
TP: HMvDR sel: Em, gen_sel: GENEtaBC
<ROOT.TNtuple object ("HMvDR_Em_GENEtaBC_reso") at 0x7fe366dd6c70>
TP: HMvDRshapeDrCalib sel: Em, gen_sel: GENEtaBC
<ROOT.TNtuple object ("HMvDRshapeDrCalib_Em_GENEtaBC_reso") at 0x7fe3777d47a0>
TP: HMvDRshapeDrCalibNew sel: Em, gen_sel: GENEtaBC
<ROOT.TNtuple object ("HMvDRshapeDrCalibNew_Em_GENEtaBC_reso") at 0x7fe366ddab50>
TP: HMvDREmInt sel: Em, gen_sel: GENEtaBC
<ROOT.TNtuple object ("HMvDREmInt_Em_GENEtaBC_reso") at 0x7fe3777d6aa0>

------ SAMPLE: ele-V9, PU: PU200
TP: EG sel: EGq5, gen_sel: GENEtaBC
<ROOT.TNtuple object ("EG_EGq5_GENEtaBC_reso") at 0x7fe3778d4bb0>
TP: EG sel: EGq4, gen_sel: GENEtaBC
<ROOT.TNtuple object ("EG_EGq4_GENEtaBC_reso") at 0x7fe3777e6b00>
TP: HMvDR sel: Em, gen_sel: GENEtaBC
<ROOT.TNt

In [10]:
tree_data['pt_resp'] = tree_data.cl_pt/tree_data.gen_pt

In [11]:
tree_data.iloc[:10]

,gen_energy,gen_eta,cl_eta,gen_pt,cl_energy,cl_pt,smp,pu,tp,gen_sel,tp_sel,pt_resp
0,414.341431,2.302596,2.298145,82.046928,389.717102,77.508301,ele-V9,PU0,EG,GENEtaBC,EGq5,0.944683
1,414.341431,-2.302596,-2.309253,82.046928,405.955292,79.863449,ele-V9,PU0,EG,GENEtaBC,EGq5,0.973387
2,174.102966,-1.866704,-1.853636,52.586536,181.803467,55.600338,ele-V9,PU0,EG,GENEtaBC,EGq5,1.057311
3,174.102966,1.866704,1.882819,52.586536,175.800598,52.289185,ele-V9,PU0,EG,GENEtaBC,EGq5,0.994345
4,67.216339,2.257846,2.260353,13.906287,54.158943,11.177411,ele-V9,PU0,EG,GENEtaBC,EGq5,0.803767
5,67.216339,-2.257846,-2.257057,13.906287,55.021313,11.392068,ele-V9,PU0,EG,GENEtaBC,EGq5,0.819203
6,50.760242,-1.645679,-1.655698,18.879072,39.678520,14.620766,ele-V9,PU0,EG,GENEtaBC,EGq5,0.774443
7,50.760242,1.645679,1.641782,18.879072,50.778862,18.954420,ele-V9,PU0,EG,GENEtaBC,EGq5,1.003991
8,104.629578,2.160485,2.149332,23.804842,103.954308,23.909388,ele-V9,PU0,EG,GENEtaBC,EGq5,1.004392
9,82.603737,-2.105754,-2.099315,19.820812,83.618134,20.190010,ele-V9,PU0,EG,GENEtaBC,EGq5,1.018627


In [12]:
# %load ./python/utilities.py
import uuid
import ROOT
from drawingTools import draw
import math

def effSigma(hist):
    xaxis = hist.GetXaxis()
    nb = xaxis.GetNbins()
    if nb < 10:
        print "effsigma: Not a valid histo. nbins = {}".format(nb)
        return -1

    bwid = xaxis.GetBinWidth(1)
    if bwid == 0:
        print "effsigma: Not a valid histo. bwid = {}".format(bwid)
        return -1

    xmax = xaxis.GetXmax()
    xmin = xaxis.GetXmin()
    ave = hist.GetMean()
    rms = hist.GetRMS()

#     print 'xmax: {}, xmin: {}, ave: {}, rms: {}'.format(xmax, xmin, ave, rms)

    total=0.
    for i in range(0, nb+2):
        total+=hist.GetBinContent(i)

    if total < 100.:
        print "effsigma: Too few entries {}".format(total)
        return -1

    ierr = 0
    ismin = 999

    rlim = 0.683*total
    # Set scan size to +/- rms
    nrms = int(rms/(bwid))
    if nrms > nb/10:
        # could be tuned
        nrms = nb/10

    widmin = 9999999.
    # scan the window center
    for iscan in range(-nrms, nrms+1):
        ibm = int((ave-xmin)/bwid+1+iscan)
        x = (ibm-0.5)*bwid+xmin;
        xj = x
        xk = x
        jbm = ibm;
        kbm = ibm;
        bin = hist.GetBinContent(ibm)
        total = bin
        for j in range(1, nb):
            if jbm < nb:
                jbm += 1
                xj += bwid
                bin = hist.GetBinContent(jbm)
                total += bin
                if total > rlim:
                    break
            else:
                ierr = 1
            if kbm > 0:
                kbm -= 1
                xk -= bwid
                bin = hist.GetBinContent(kbm)
                total += bin;
                if total > rlim:
                    break
            else:
                ierr = 1
        dxf = 0
        if bin != 0:
            dxf = (total-rlim)*bwid/bin
        wid = (xj-xk+bwid-dxf)*0.5
        if wid < widmin:
            widmin=wid
            ismin=iscan

    if ismin == nrms or ismin == -nrms:
        ierr = 3
    if ierr != 0:
        print "effsigma: Error of type {}".format(ierr)

    return widmin


def quantiles(yswz, zeroSuppress=True):
    ys = [y for y in yswz if y > 0] if zeroSuppress else yswz[:]
    if len(ys) < 3: return (0,0,0)
    ys.sort()
    ny = len(ys)
    median = ys[ny/2]
   #if ny > 400e9:
   #    u95 = ys[min(int(ceil(ny*0.975)),ny-1) ]
   #    l95 = ys[int(floor(ny*0.025))]
   #    u68 = 0.5*(median+u95)
   #    l68 = 0.5*(median+l95)
    if ny > 20:
        u68 = ys[min(int(math.ceil(ny*0.84)),ny-1) ]
        l68 = ys[int(math.floor(ny*0.16))]
    else:
        rms = math.sqrt(sum((y-median)**2 for y in ys)/ny)
        u68 = median + rms
        l68 = median - rms
    return (median,l68,u68)


def gausstailfit_wc(name, project_hist, bin_limits):
    global cache
    if cache is not None and not cache[(cache.h_name == name) & (cache.bin_limits == bin_limits)].empty:
        print 'READ cached fit results h_name: {}, bin_limits: {}'.format(name, bin_limits)
#         print cache[(cache.h_name == name) & (cache.bin_limits == bin_limits)].results
        return cache[(cache.h_name == name) & (cache.bin_limits == bin_limits)].results.values[0]

    max_bin = project_hist.GetMaximumBin()
    max_value = project_hist.GetBinCenter(max_bin)
    rms_value = project_hist.GetRMS()
    max_y = project_hist.GetMaximum()
#     print max_bin, max_value, rms_value

    def gausstail(x, p):
        #         // [Constant] * ROOT::Math::crystalball_function(x, [Alpha], [N], [Sigma], [Mean])
        return p[0] * ROOT.Math.crystalball_function(x[0], p[3], p[4], p[2], p[1])

    fitf = ROOT.TF1('gausstail', gausstail, -1.5, 1.5, 5)
    fitf.SetParNames('norm', 'mean', 'sigma', 'alpha', 'n')
#     fitf.FixParameter(0, 1.)
    fitf.SetParLimits(1, max_value-0.04, max_value+0.04)

    fitf.SetParameters(max_y, max_value, rms_value, 1, 1)
    draw([project_hist], labels=['fit'], text=name)
    stuff.append(fitf)
    project_hist.Draw("same")
#     c.Draw()
#     print '   name: {}, y_max = {}, max_value = {}, RMS = {}'.format(name, max_y, max_value, rms_value)
    result = project_hist.Fit('gausstail', 'QERLS+')
    result.Print()
#     print '   norm = {}, reso_mean = {}, reso_sigma = {}, alpha = {}, n = {}'.format(result.GetParams()[0], result.GetParams()[1], result.GetParams()[2], result.GetParams()[3], result.GetParams()[4])
#     func = project_hist.GetFunction("gaus")
    # print '   NDF = {}, chi2 = {}, prob = {}'.format(fitf.GetNDF(), fitf.GetChisquare(), fitf.GetProb())

    if cache is not None:
        cache = cache.append({'h_name': name,
                              'bin_limits': bin_limits,
                              'results': result}, ignore_index=True)
#         print cache

    return result

def effective_sigma_energy(project_hist):
    eff_sigma = effSigma(project_hist)
    bin_values = [project_hist.GetBinContent(bin) for bin in range(1, project_hist.GetNbinsX()+1)]
    return (eff_sigma,)


def gausstailfit_energy(project_hist):
    max_bin = project_hist.GetMaximumBin()
    max_value = project_hist.GetBinCenter(max_bin)
    rms_value = project_hist.GetRMS()
    max_y = project_hist.GetMaximum()

    def gausstail(x, p):
        return p[0] * ROOT.Math.crystalball_function(x[0], p[3], p[4], p[2], p[1])

    fitf = ROOT.TF1('gausstail', gausstail, -1.0, 0.5, 5)
    fitf.SetParNames('norm', 'mean', 'sigma', 'alpha', 'n')
#     fitf.FixParameter(0, 1.)
    fitf.SetParLimits(1, max_value-0.04, max_value+0.04)
    fitf.SetParameters(max_y, max_value, rms_value, 1, 1)
    stuff.append(fitf)
    project_hist.Draw()
#     c.Draw()
#     print '   y_max = {}, max_value = {}, RMS = {}'.format(max_y, max_value, rms_value)
    result = project_hist.Fit('gausstail', 'QERLS+')
    result.Print()
#     print '   norm = {}, reso_mean = {}, reso_sigma = {}, alpha = {}, n = {}'.format(result.GetParams()[0],
#                                                                                      result.GetParams()[1],
#                                                                                      result.GetParams()[2],
#                                                                                      result.GetParams()[3],
#                                                                                      result.GetParams()[4])
    return result.GetParams()[0], result.GetParams()[1], result.GetParams()[2], result.GetParams()[3], result.GetParams()[4]


def gausstailfit_ptresp(project_hist, x_low=0., x_high=1.2):
    max_bin = project_hist.GetMaximumBin()
    max_value = project_hist.GetBinCenter(max_bin)
    rms_value = project_hist.GetRMS()
    max_y = project_hist.GetMaximum()

    eff_sigma = effSigma(project_hist)

    prob = np.array([0.001, 0.999])

    q = np.array([0., 1.2])
    y = project_hist.GetQuantiles(2,q,prob)
    x_low = q[0]
    x_high = q[1]

    def gausstail(x, p):
        return p[0] * ROOT.Math.crystalball_function(x[0], p[3], p[4], p[2], p[1])
    
    print  x_low, x_high
    fitf = ROOT.TF1('gausstail', gausstail, x_low, x_high, 5)

    fitf.SetParNames('norm', 'mean', 'sigma', 'alpha', 'n')
#     fitf.FixParameter(0, 1.)
    fitf.SetParLimits(1, max_value-eff_sigma, max_value+eff_sigma)
    fitf.SetParameters(max_y, max_value, eff_sigma, 1, 1)
    stuff.append(fitf)
    project_hist.Draw()

    
    #     c.Draw()
#     print '   y_max = {}, max_value = {}, RMS = {}'.format(max_y, max_value, rms_value)
    result = project_hist.Fit('gausstail', 'QERLS+')
    result.Print()
    print 'CHi2 prob: {}'.format(fitf.GetProb())
#     print '   norm = {}, reso_mean = {}, reso_sigma = {}, alpha = {}, n = {}'.format(result.GetParams()[0],
#                                                                                      result.GetParams()[1],
#                                                                                      result.GetParams()[2],
#                                                                                      result.GetParams()[3],
#                                                                                      result.GetParams()[4])
    return result.GetParams()[0], result.GetParams()[1], result.GetParams()[2], result.GetParams()[3], result.GetParams()[4], fitf.GetProb()




def computeResolution(histo2d,
                      bin_limits,
                      y_axis_range,
                      fit_function,
                      result_index,
                      cache=None):
    global stuff

    def get_results(histo_name,
                    project_hist,
                    bin_limits,
                    fit_function,
                    cache=None):
        if cache is not None:
            if not cache[(cache.h_name == histo_name) &
                         (cache.bin_limits == bin_limits) &
                         (cache.fit_function == fit_function)].empty:
                print 'READ cached fit results h_name: {}, bin_limits: {}, fit_function: {}'.format(histo_name,
                                                                                                    bin_limits,
                                                                                                    fit_function)
                return cache[(cache.h_name == histo_name) &
                            (cache.bin_limits == bin_limits) &
                            (cache.fit_function == fit_function)].results.values[0]
            else:
                print "No ENTRY in CACHE"
                result = fit_function(project_hist)
                cache.loc[cache.shape[0]+1] = {'h_name': histo_name,
                                      'bin_limits': bin_limits,
                                      'fit_function': fit_function,
                                      'results': result}
                return result
        return fit_function(project_hist)



    h2d = histo2d.Clone()
    h2d.GetYaxis().SetRangeUser(y_axis_range[0], y_axis_range[1])

    x, y, ex_l, ex_h, ey_l, ey_h = [], [], [], [], [], []
    print '-----------------------'
    for x_bin_low, x_bin_high in bin_limits:
        y_proj = h2d.ProjectionY(uuid.uuid4().hex[:6]+'_y', x_bin_low, x_bin_high)
        stuff.append(y_proj)
        x_low = h2d.GetXaxis().GetBinLowEdge(x_bin_low)
        x_high = h2d.GetXaxis().GetBinUpEdge(x_bin_high)
#         print 'x_low: {} x_high: {}'.format(x_low, x_high)
        draw([y_proj], labels=['fit'], text='BIN: ({}, {}) = ({}, {}) GeV, RES: {}'.format(
                                            x_bin_low, x_bin_high, x_low, x_high, 0))

        fit_result = get_results(histo2d.GetName(),
                                 y_proj,
                                 (x_bin_low, x_bin_high),
                                 fit_function,
                                 cache)
#         draw([y_proj], labels=['fit'], text='BIN: ({}, {}) = ({}, {}) GeV, RES: {}'.format(
#                                             x_bin_low, x_bin_high, x_low, x_high, fit_result[result_index]))

        h2d.SetAxisRange(x_low, x_high)
        x_mean = h2d.GetMean()
        x.append(x_mean)
        ex_l.append(0)
        ex_h.append(0)
        y.append(fit_result[result_index])
        ey_l.append(0)
        ey_h.append(0)
    return x, y, ex_l, ex_h, ey_l, ey_h


def computeEResolution(h2d_orig,
                       bins_limits=[(3, 6), (7, 12), (13, 23), (24, 34), (35, 49), (50, 100)],
                       cache=None):
    global stuff
    h2d = h2d_orig.Clone()
    h2d.GetYaxis().SetRangeUser(-100, 100)
    x, y, ex_l, ex_h, ey_l, ey_h = [], [], [], [], [], []
    print '-----------------------'
    for x_bin_low, x_bin_high in bins_limits:
        y_proj = h2d.ProjectionY(uuid.uuid4().hex[:6]+'_y', x_bin_low, x_bin_high)
        stuff.append(y_proj)
        x_low = h2d.GetXaxis().GetBinLowEdge(x_bin_low)
        x_high = h2d.GetXaxis().GetBinUpEdge(x_bin_high)
#         print 'x_low: {} x_high: {}'.format(x_low, x_high)
#         fit_result = gausstailfit(h2d_orig.GetName(), y_proj)
        fit_result = gausstailfit_wc(h2d_orig.GetName(), y_proj, (x_bin_low, x_bin_high))
        h2d.SetAxisRange(x_low, x_high)
        x_mean = h2d.GetMean()
#         print 'mean: {}'.format(x_mean)
    #     x_value = h2d.GetXaxis().GetBinCenter(x_bin)
    #     x_err_min = x_value - h2d.GetXaxis().GetBinLowEdge(x_bin)
    #     x_err_plus = h2d.GetXaxis().GetBinUpEdge(x_bin) - x_value

        x.append(x_mean)
        ex_l.append(0)
        ex_h.append(0)
        y.append(fit_result.GetParams()[2])

#         y.append(fit_result.GetParams()[2]/x_mean)
        ey_l.append(0)
        ey_h.append(0)
    return x, y, ex_l, ex_h, ey_l, ey_h


def computeEResolutionMean(h2d_orig,
                           bins_limits=[(3, 6), (7, 12), (13, 23), (24, 34), (35, 49), (50, 100)],
                           cache=None):
    h2d = h2d_orig.Clone()
    h2d.GetYaxis().SetRangeUser(-100, 100)
    x, y, ex_l, ex_h, ey_l, ey_h = [], [], [], [], [], []
    print '-----------------------'
    for x_bin_low, x_bin_high in bins_limits:
        y_proj = h2d.ProjectionY(uuid.uuid4().hex[:6]+'_y', x_bin_low, x_bin_high)
        stuff.append(y_proj)
        x_low = h2d.GetXaxis().GetBinLowEdge(x_bin_low)
        x_high = h2d.GetXaxis().GetBinUpEdge(x_bin_high)
        print 'x_low: {} x_high: {}'.format(x_low, x_high)
#         fit_result = gausstailfit(h2d_orig.GetName(), y_proj)
        fit_result = gausstailfit_wc(h2d_orig.GetName(), y_proj, (x_bin_low, x_bin_high))

        h2d.SetAxisRange(x_low, x_high)
        x_mean = h2d.GetMean()
#         print 'mean: {}'.format(x_mean)
    #     x_value = h2d.GetXaxis().GetBinCenter(x_bin)
    #     x_err_min = x_value - h2d.GetXaxis().GetBinLowEdge(x_bin)
    #     x_err_plus = h2d.GetXaxis().GetBinUpEdge(x_bin) - x_value

        x.append(x_mean)
        ex_l.append(0)
        ex_h.append(0)

        y.append(fit_result.GetParams()[1])
        ey_l.append(0)
        ey_h.append(0)
    return x, y, ex_l, ex_h, ey_l, ey_h


In [13]:

# print ys

def get_gauss_avg_sigma(ys):
    (median,lo,hi) = quantiles(ys, False)
    # print median,lo,hi
    avg = median
    rms2 = (hi - lo)
    for niter in xrange(3):
        truncated = [y for y in ys if abs(y-avg) < rms2]
        if len(truncated) <= 2: 
            break
        avg = sum(truncated)/len(truncated)
        rms2 = 2*math.sqrt(sum((t-avg)**2 for t in truncated)/(len(truncated)-1))
    return avg, rms2/2


### Fill histograms

In [14]:
class ResoPlot(histos.ResoHistos):
    def __init__(self,  name, root_file=None, debug=False):
        self.name = name

        self.pt_resp_binned = []
        self.h_ptRespVeta = ROOT.TH2F(
            name+'_ptRespVeta', 'Pt resp. v eta; |#eta_{GEN}|; p_{T}^{L1}/p_{T}^{GEN};', 50, 1.4, 3, 100, 0, 2)
       
        histos.BaseResoHistos.__init__(self, name, root_file)        
        
    def fill_df(self, sel_df):
        pt_bin_pitch = 10
        pt_bin_min = 5
        pt_bin_max = 100
        pt_bins = []
        rnp.fill_hist(self.h_ptRespVeta, sel_df[['gen_eta','pt_resp']])

        for x in range(pt_bin_min, pt_bin_max, pt_bin_pitch):
            pt_bins.append((x, x+pt_bin_pitch))
#         pt_bins = [(5, 10), (10, 15), (15, 20), (20, 25), (25, 30), (30, 35), (35, 40), (40, 45), (45, 50), (50, 55), (55, 60), (60, 65), (65, 70), (70, 80), (80, 90), (90, 100)]
        self.g_pt_resp_gauss_sigma = ROOT.TGraphAsymmErrors(len(pt_bins))
        self.g_pt_resp_gauss_avg = ROOT.TGraphAsymmErrors(len(pt_bins))
        self.g_pt_resp_cb_sigma = ROOT.TGraphAsymmErrors(len(pt_bins))
        self.g_pt_resp_cb_avg = ROOT.TGraphAsymmErrors(len(pt_bins))
        self.g_pt_resp_cb_chi2prob = ROOT.TGraphAsymmErrors(len(pt_bins))

        self.g_pt_resp_eff_sigma = ROOT.TGraphAsymmErrors(len(pt_bins))
        self.g_pt_resp_rms = ROOT.TGraphAsymmErrors(len(pt_bins))
        self.g_pt_resp_median = ROOT.TGraphAsymmErrors(len(pt_bins))
        
        for ipoint, (pt_min_low, pt_bin_high) in enumerate(pt_bins):
#             print pt_min_low, pt_bin_high
            res_selection = sel_df[(sel_df.gen_pt > pt_min_low) & (sel_df.gen_pt < pt_bin_high)].pt_resp
            avg_gauss, sigma_gauss = get_gauss_avg_sigma(res_selection.values)
            h_pt_resp_name = 'h_ptresp_bin_{}_{}_{}'.format(pt_min_low, pt_bin_high, self.name)
            h_pt_resp_title = 'h_ptresp_bin_{}_{}_{}'.format(pt_min_low, pt_bin_high, self.name)

            h_pt_resp_bin = ROOT.TH1F(h_pt_resp_name, h_pt_resp_title, 100, 0, 2)
            rnp.fill_hist(h_pt_resp_bin, res_selection)    
            self.pt_resp_binned.append(h_pt_resp_bin)
            c_pt_resp = newCanvas(h_pt_resp_name)
#             c = ROOT.TCanvas()
#             h_pt_resp_bin.Draw()
#             c.Draw()
            eff_sigma = effSigma(h_pt_resp_bin)
#             eff_sigma = 1
    #             print eff_sigma
            cb_fit_results = gausstailfit_ptresp(h_pt_resp_bin)
            c_pt_resp.Draw()
        
            median = res_selection.median()

            
            print 'BIN: {},{} avg: {}, sigma_gauss: {}, mean_cb: {}, sigma_cb: {}, sigma_eff: {}'.format(pt_min_low, pt_bin_high, 
                                                                                             avg_gauss, sigma_gauss,
                                                                                             cb_fit_results[1], cb_fit_results[2],
                                                                                             eff_sigma)

            bin_center = (pt_bin_high+pt_min_low)/2.
            bin_half_witdh = (pt_bin_high-pt_min_low)/2.

            self.g_pt_resp_gauss_sigma.SetPoint(ipoint, bin_center, sigma_gauss)
            self.g_pt_resp_gauss_sigma.SetPointError(ipoint, bin_half_witdh, bin_half_witdh, 0, 0)

            self.g_pt_resp_gauss_avg.SetPoint(ipoint, bin_center, avg_gauss)
            self.g_pt_resp_gauss_avg.SetPointError(ipoint, bin_half_witdh, bin_half_witdh, 0, 0)

            self.g_pt_resp_cb_sigma.SetPoint(ipoint, bin_center, cb_fit_results[2])
            self.g_pt_resp_cb_sigma.SetPointError(ipoint, bin_half_witdh, bin_half_witdh, 0, 0)

            self.g_pt_resp_cb_avg.SetPoint(ipoint, bin_center, cb_fit_results[1])
            self.g_pt_resp_cb_avg.SetPointError(ipoint, bin_half_witdh, bin_half_witdh, 0, 0)

            self.g_pt_resp_cb_chi2prob.SetPoint(ipoint, bin_center, cb_fit_results[-1])
            self.g_pt_resp_cb_chi2prob.SetPointError(ipoint, bin_half_witdh, bin_half_witdh, 0, 0)
            
            self.g_pt_resp_eff_sigma.SetPoint(ipoint, bin_center, eff_sigma)
            self.g_pt_resp_eff_sigma.SetPointError(ipoint, bin_half_witdh, bin_half_witdh, 0, 0)

            self.g_pt_resp_rms.SetPoint(ipoint, bin_center, h_pt_resp_bin.GetRMS())
            self.g_pt_resp_rms.SetPointError(ipoint, bin_half_witdh, bin_half_witdh, 0, 0)

            self.g_pt_resp_median.SetPoint(ipoint, bin_center, median)
            self.g_pt_resp_median.SetPointError(ipoint, bin_half_witdh, bin_half_witdh, 0, 0)

            
class HWrapper(object):
    def __init__(self, histo):
        self.histo = histo
    
    def get(self, debug=False):
        return self.histo


In [15]:
ROOT.gStyle.SetOptFit(True)

In [16]:
hplot = HPlot(samples, labels_dict)

for smp in samples:
    print ""
    print '------ SAMPLE: {}, PU: {}'.format(smp.type, smp.label)
    for tp, tp_sel, gen_sel in tp_selections:
        print 'TP: {} sel: {}, gen_sel: {}'.format(tp, tp_sel, gen_sel)
        
        if 'TkEle' in tp and 'photon' in smp.type:
            continue

        hres = ResoPlot(name='{}_{}_{}_{}_{}'.format(smp.type, smp.label, tp, tp_sel, gen_sel))

        sel_df = tree_data[(tree_data.smp == smp.type) &
           (tree_data.pu == smp.label) &
           (tree_data.tp == tp) &
           (tree_data.tp_sel == tp_sel) &
           (tree_data.gen_sel == gen_sel)]



        hres.fill_df(sel_df)

        hplot.data = hplot.data.append({'sample': smp.type,
                               'pu': smp.label,
                               'tp': tp,
                               'tp_sel': tp_sel,
                               'gen_sel': gen_sel,
                               'classtype': ResoPlot,
                               'histo': HWrapper(hres),},
                               ignore_index=True)
   


------ SAMPLE: ele-V9, PU: PU0
TP: EG sel: EGq5, gen_sel: GENEtaBC
0.19329 1.65671
CHi2 prob: 2.95396532381e-21
BIN: 5,15 avg: 0.883753017757, sigma_gauss: 0.150383339099, mean_cb: 0.905908094293, sigma_cb: 0.214149246884, sigma_eff: 0.203120560748
0.385426666667 1.52790666667
CHi2 prob: 5.80537226589e-14
BIN: 15,25 avg: 0.92548223412, sigma_gauss: 0.104385878618, mean_cb: 0.931725310805, sigma_cb: 0.146562900568, sigma_eff: 0.141490646766
0.42055 1.334725
CHi2 prob: 1.61529601795e-08
BIN: 25,35 avg: 0.949094210767, sigma_gauss: 0.0785256369271, mean_cb: 0.95269219222, sigma_cb: 0.103770329053, sigma_eff: 0.10824826087
0.4101 1.22993333333
CHi2 prob: 2.22032389645e-10
BIN: 35,45 avg: 0.964571264079, sigma_gauss: 0.0606895189922, mean_cb: 0.96778389213, sigma_cb: 0.0809359022704, sigma_eff: 0.0850700665188
0.42097 1.194515
CHi2 prob: 0.000504628741944
BIN: 45,55 avg: 0.976609715614, sigma_gauss: 0.0526091369778, mean_cb: 0.977744219131, sigma_cb: 0.0682769941073, sigma_eff: 0.072875045

CHi2 prob: 3.04703736749e-08
BIN: 35,45 avg: 0.964604584281, sigma_gauss: 0.0612412715635, mean_cb: 0.967798072489, sigma_cb: 0.081142911777, sigma_eff: 0.0861443080357
0.12258 1.1798968
CHi2 prob: 3.80661449076e-05
BIN: 45,55 avg: 0.976528683945, sigma_gauss: 0.0527448852508, mean_cb: 0.977822589735, sigma_cb: 0.0680268935149, sigma_eff: 0.0736927906977
0.12182 1.1622725
CHi2 prob: 0.00214588442964
BIN: 55,65 avg: 0.983002523164, sigma_gauss: 0.0463074465402, mean_cb: 0.984043584049, sigma_cb: 0.059441133542, sigma_eff: 0.0655644034707
0.05934 1.14355333333
CHi2 prob: 0.519469989727
BIN: 65,75 avg: 0.988835325521, sigma_gauss: 0.0409458292104, mean_cb: 0.989528781653, sigma_cb: 0.0524087105877, sigma_eff: 0.0577457317073
0.0457466666667 1.14570545455
CHi2 prob: 0.522807859263
BIN: 75,85 avg: 0.991780972694, sigma_gauss: 0.0379924852724, mean_cb: 0.99241535047, sigma_cb: 0.0489904749624, sigma_eff: 0.0537943379791
0.06478 1.13340666667
CHi2 prob: 7.95838144496e-08
BIN: 85,95 avg: 0.994

0.29488 1.22341333333
CHi2 prob: 1.10906778819e-05
BIN: 65,75 avg: 0.993465337822, sigma_gauss: 0.0447209957669, mean_cb: 0.994291715709, sigma_cb: 0.0566988188965, sigma_eff: 0.0643770769231
0.28173 1.183308
CHi2 prob: 0.000146491146468
BIN: 75,85 avg: 0.995102428991, sigma_gauss: 0.0408178039358, mean_cb: 0.995728893563, sigma_cb: 0.0516154660508, sigma_eff: 0.0592408265802
0.235872 1.18064
CHi2 prob: 0.0045488027986
BIN: 85,95 avg: 0.994356773285, sigma_gauss: 0.0372175810459, mean_cb: 0.994586300549, sigma_cb: 0.046953299132, sigma_eff: 0.0538794202899
0.26925 1.16075
CHi2 prob: 3.1384596478e-06
BIN: 95,105 avg: 0.993726275875, sigma_gauss: 0.0353452940998, mean_cb: 0.994626786186, sigma_cb: 0.0459797219484, sigma_eff: 0.0505401960784
TP: HMvDRshapeDrCalib sel: Em, gen_sel: GENEtaBC
0.158466666667 1.85486666667
CHi2 prob: 6.56698059084e-11
BIN: 5,15 avg: 0.961576247688, sigma_gauss: 0.190222478622, mean_cb: 0.980437581867, sigma_cb: 0.261575348977, sigma_eff: 0.252556229508
0.34931

In [ ]:
#  self.g_pt_resp_gauss_sigma = ROOT.TGraphAsymmErrors(len(pt_bins))
#         self.g_pt_resp_gauss_avg = ROOT.TGraphAsymmErrors(len(pt_bins))
#         self.g_pt_resp_cb_sigma = ROOT.TGraphAsymmErrors(len(pt_bins))
#         self.g_pt_resp_cb_avg = ROOT.TGraphAsymmErrors(len(pt_bins))
#         self.g_pt_resp_eff_sigma = ROOT.TGraphAsymmErrors(len(pt_bins))
#         self.g_pt_resp_rms = R

In [ ]:
dm = DrawMachine(rleg_config)
hsets, labels, text = hplot.get_histo(ResoPlot, 'photons-V9', 'PU0', ['EG'], ['EGq5'], 'GENEtaBC', debug=False)

dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['gauss.'])
dm.addHistos([his.g_pt_resp_cb_sigma for his in hsets], ['CB'])
dm.addHistos([his.g_pt_resp_eff_sigma for his in hsets], ['eff'])
dm.addHistos([his.g_pt_resp_rms for his in hsets], ['rms'])

# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew for his in hsets], ['new'])
# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew2 for his in hsets], ['effSigma'])

dm.draw(text=text, options='COLZ', y_min=0., y_max=0.1)



In [ ]:
dm = DrawMachine(rleg_config)
hsets, labels, text = hplot.get_histo(ResoPlot, 'photons-V9', 'PU0', ['EG'], ['EGq5'], 'GENEtaBC', debug=False)

dm.addHistos([his.g_pt_resp_gauss_avg for his in hsets], ['gauss.'])
dm.addHistos([his.g_pt_resp_cb_avg for his in hsets], ['CB'])

# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew for his in hsets], ['new'])
# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew2 for his in hsets], ['effSigma'])

dm.draw(text=text, options='COLZ', y_min=0.7, y_max=1.2)


In [ ]:
dm = DrawMachine(rleg_config)
hsets, labels, text = hplot.get_histo(ResoPlot, 'photons-V9', 'PU200', ['EG'], ['EGq5'], 'GENEtaBC', debug=False)

dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['gauss.'])
dm.addHistos([his.g_pt_resp_cb_sigma for his in hsets], ['CB'])
dm.addHistos([his.g_pt_resp_eff_sigma for his in hsets], ['eff'])
dm.addHistos([his.g_pt_resp_rms for his in hsets], ['rms'])

# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew for his in hsets], ['new'])
# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew2 for his in hsets], ['effSigma'])

dm.draw(text=text, options='COLZ', y_min=0., y_max=0.1)



In [ ]:
dm = DrawMachine(rleg_config)
hsets, labels, text = hplot.get_histo(ResoPlot, 'photons-V9', 'PU0', ['EG'], ['EGq5'], 'GENEtaBC', debug=False)

dm.addHistos([his.g_pt_resp_gauss_avg for his in hsets], ['gauss.'])
dm.addHistos([his.g_pt_resp_cb_avg for his in hsets], ['CB'])
dm.addHistos([his.g_pt_resp_median for his in hsets], ['median'])

# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew for his in hsets], ['new'])
# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew2 for his in hsets], ['effSigma'])

dm.draw(text=text, options='COLZ', y_min=0.7, y_max=1.2)


In [ ]:
dm = DrawMachine(rleg_config)
hsets, labels, text = hplot.get_histo(ResoPlot, 'photons-V9', 'PU200', ['EG'], ['EGq5'], 'GENEtaBC', debug=False)

dm.addHistos([his.g_pt_resp_gauss_avg for his in hsets], ['gauss.'])
dm.addHistos([his.g_pt_resp_cb_avg for his in hsets], ['CB'])
dm.addHistos([his.g_pt_resp_median for his in hsets], ['median'])

# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew for his in hsets], ['new'])
# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew2 for his in hsets], ['effSigma'])

dm.draw(text=text, options='COLZ', y_min=0.7, y_max=1.2)


In [ ]:
dm = DrawMachine(rleg_config)
hsets, labels, text = hplot.get_histo(ResoPlot, 'ele-V9', 'PU0', ['EG'], ['EGq5'], 'GENEtaBC', debug=False)

dm.addHistos([his.g_pt_resp_gauss_avg for his in hsets], ['gauss.'])
dm.addHistos([his.g_pt_resp_cb_avg for his in hsets], ['CB'])
dm.addHistos([his.g_pt_resp_median for his in hsets], ['median'])

# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew for his in hsets], ['new'])
# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew2 for his in hsets], ['effSigma'])

dm.draw(text=text, options='COLZ', y_min=0.7, y_max=1.2)


In [ ]:
dm = DrawMachine(rleg_config)
hsets, labels, text = hplot.get_histo(ResoPlot, 'ele-V9', 'PU200', ['EG'], ['EGq5'], 'GENEtaBC', debug=False)

dm.addHistos([his.g_pt_resp_gauss_avg for his in hsets], ['gauss.'])
dm.addHistos([his.g_pt_resp_cb_avg for his in hsets], ['CB'])
dm.addHistos([his.g_pt_resp_median for his in hsets], ['median'])

# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew for his in hsets], ['new'])
# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew2 for his in hsets], ['effSigma'])

dm.draw(text=text, options='COLZ', y_min=0.7, y_max=1.2)


In [ ]:
dm = DrawMachine(rleg_config)
hsets, labels, text = hplot.get_histo(ResoPlot, 'photons-V9', 'PU0', ['EGBRL'], ['LooseTkID'], 'GENEtaF', debug=False)

dm.addHistos([his.g_pt_resp_gauss_avg for his in hsets], ['gauss.'])
dm.addHistos([his.g_pt_resp_cb_avg for his in hsets], ['CB'])
dm.addHistos([his.g_pt_resp_median for his in hsets], ['median'])

# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew for his in hsets], ['new'])
# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew2 for his in hsets], ['effSigma'])

dm.draw(text=text, options='COLZ', y_min=0.7, y_max=1.2)


In [ ]:
dm = DrawMachine(rleg_config)
hsets, labels, text = hplot.get_histo(ResoPlot, 'photons-V9', 'PU200', ['EGBRL'], ['LooseTkID'], 'GENEtaF', debug=False)

dm.addHistos([his.g_pt_resp_gauss_avg for his in hsets], ['gauss.'])
dm.addHistos([his.g_pt_resp_cb_avg for his in hsets], ['CB'])
dm.addHistos([his.g_pt_resp_median for his in hsets], ['median'])

# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew for his in hsets], ['new'])
# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew2 for his in hsets], ['effSigma'])

dm.draw(text=text, options='COLZ', y_min=0.7, y_max=1.2)


In [ ]:
dm = DrawMachine(rleg_config)
hsets, labels, text = hplot.get_histo(ResoPlot, 'ele-V9', 'PU0', ['EGBRL'], ['LooseTkID'], 'GENEtaF', debug=False)

dm.addHistos([his.g_pt_resp_gauss_avg for his in hsets], ['gauss.'])
dm.addHistos([his.g_pt_resp_cb_avg for his in hsets], ['CB'])
dm.addHistos([his.g_pt_resp_median for his in hsets], ['median'])

# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew for his in hsets], ['new'])
# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew2 for his in hsets], ['effSigma'])

dm.draw(text=text, options='COLZ', y_min=0.7, y_max=1.2)


In [ ]:
dm = DrawMachine(rleg_config)
hsets, labels, text = hplot.get_histo(ResoPlot, 'ele-V9', 'PU200', ['EGBRL'], ['LooseTkID'], 'GENEtaF', debug=False)

dm.addHistos([his.g_pt_resp_gauss_avg for his in hsets], ['gauss.'])
dm.addHistos([his.g_pt_resp_cb_avg for his in hsets], ['CB'])
dm.addHistos([his.g_pt_resp_median for his in hsets], ['median'])

# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew for his in hsets], ['new'])
# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew2 for his in hsets], ['effSigma'])

dm.draw(text=text, options='COLZ', y_min=0.7, y_max=1.2)


In [ ]:
dm = DrawMachine(rleg_config)
hsets, labels, text = hplot.get_histo(ResoPlot, 'ele-V9', 'PU0', ['EG'], ['EGq5'], 'GENEtaBC', debug=False)

dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['gauss.'])
dm.addHistos([his.g_pt_resp_cb_sigma for his in hsets], ['CB'])
dm.addHistos([his.g_pt_resp_eff_sigma for his in hsets], ['eff'])
dm.addHistos([his.g_pt_resp_rms for his in hsets], ['rms'])

# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew for his in hsets], ['new'])
# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew2 for his in hsets], ['effSigma'])

dm.draw(text=text, options='COLZ', y_min=0., y_max=0.3)


In [ ]:
dm = DrawMachine(rleg_config)
hsets, labels, text = hplot.get_histo(ResoPlot, 'ele-V9', 'PU0', ['EG'], ['EGq5'], 'GENEtaBC', debug=False)

dm.addHistos([his.g_pt_resp_gauss_avg for his in hsets], ['gauss.'])
dm.addHistos([his.g_pt_resp_cb_avg for his in hsets], ['CB'])

# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew for his in hsets], ['new'])
# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew2 for his in hsets], ['effSigma'])

dm.draw(text=text, options='COLZ', y_min=0.7, y_max=1.2)



In [ ]:
dm = DrawMachine(rleg_config)
hsets, labels, text = hplot.get_histo(ResoPlot, 'ele-V9', 'PU200', ['EG'], ['EGq5'], 'GENEtaBC', debug=False)

dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['gauss.'])
dm.addHistos([his.g_pt_resp_cb_sigma for his in hsets], ['CB'])
dm.addHistos([his.g_pt_resp_eff_sigma for his in hsets], ['eff'])
dm.addHistos([his.g_pt_resp_rms for his in hsets], ['rms'])

# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew for his in hsets], ['new'])
# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew2 for his in hsets], ['effSigma'])

dm.draw(text=text, options='COLZ', y_min=0., y_max=0.3)


In [ ]:
dm = DrawMachine(rleg_config)
hsets, labels, text = hplot.get_histo(ResoPlot, 'ele-V9', 'PU200', ['EG'], ['EGq5'], 'GENEtaDE', debug=False)

dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['gauss.'])
dm.addHistos([his.g_pt_resp_cb_sigma for his in hsets], ['CB'])
dm.addHistos([his.g_pt_resp_eff_sigma for his in hsets], ['eff'])
dm.addHistos([his.g_pt_resp_rms for his in hsets], ['rms'])

# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew for his in hsets], ['new'])
# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew2 for his in hsets], ['effSigma'])

dm.draw(text=text, options='COLZ', y_min=0., y_max=0.3)


In [ ]:
dm = DrawMachine(rleg_config)
hsets, labels, text = hplot.get_histo(ResoPlot, 'ele-V9', 'PU200', ['EG'], ['EGq5'], 'GENEtaD', debug=False)

dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['gauss.'])
dm.addHistos([his.g_pt_resp_cb_sigma for his in hsets], ['CB'])
dm.addHistos([his.g_pt_resp_eff_sigma for his in hsets], ['eff'])
dm.addHistos([his.g_pt_resp_rms for his in hsets], ['rms'])

# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew for his in hsets], ['new'])
# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew2 for his in hsets], ['effSigma'])

dm.draw(text=text, options='COLZ', y_min=0., y_max=0.3)


In [ ]:
dm = DrawMachine(rleg_config)
hsets, labels, text = hplot.get_histo(ResoPlot, 'ele-V9', 'PU200', ['EGBRL'], ['LooseTkID'], 'GENEtaF', debug=False)

dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['gauss.'])
dm.addHistos([his.g_pt_resp_cb_sigma for his in hsets], ['CB'])
dm.addHistos([his.g_pt_resp_eff_sigma for his in hsets], ['eff'])
dm.addHistos([his.g_pt_resp_rms for his in hsets], ['rms'])

# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew for his in hsets], ['new'])
# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew2 for his in hsets], ['effSigma'])

dm.draw(text=text, options='COLZ', y_min=0., y_max=0.3)


In [ ]:
dm = DrawMachine(rleg_config)
hsets, labels, text = hplot.get_histo(ResoPlot, 'ele-V9', 'PU0', ['EG'], ['EGq5'], 'GENEtaBC', debug=False)

dm.addHistos([his.g_pt_resp_gauss_avg for his in hsets], ['gauss.'])
dm.addHistos([his.g_pt_resp_cb_avg for his in hsets], ['CB'])

# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew for his in hsets], ['new'])
# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew2 for his in hsets], ['effSigma'])

dm.draw(text=text, options='COLZ', y_min=0.7, y_max=1.2)



In [ ]:
dm = DrawMachine(rleg_config)
hsets, labels, text = hplot.get_histo(ResoPlot, ['ele-V9', 'photons-V9'], 'PU200', ['EG'], ['EGq5'], 'GENEtaBC', debug=False)

# dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['gauss.'])
dm.addHistos([his.g_pt_resp_cb_avg for his in hsets], labels)
# dm.addHistos([his.g_pt_resp_eff_sigma for his in hsets], ['eff'])
# dm.addHistos([his.g_pt_resp_rms for his in hsets], ['rms'])

# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew for his in hsets], ['new'])
# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew2 for his in hsets], ['effSigma'])

dm.draw(text=text, options='', y_min=0.8, y_max=1.2)


In [ ]:
dm = DrawMachine(rleg_config)
hsets, labels, text = hplot.get_histo(ResoPlot, ['ele-V9', 'photons-V9'], 'PU200', ['EG'], ['EGq5'], 'GENEtaBC', debug=False)

# dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['gauss.'])
dm.addHistos([his.g_pt_resp_cb_sigma for his in hsets], labels)
# dm.addHistos([his.g_pt_resp_eff_sigma for his in hsets], ['eff'])
# dm.addHistos([his.g_pt_resp_rms for his in hsets], ['rms'])

# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew for his in hsets], ['new'])
# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew2 for his in hsets], ['effSigma'])

dm.draw(text=text, options='', y_min=0., y_max=0.3)


### New heading

### CB Sigma

In [ ]:
dm = DrawMachine(rleg_config)
hsets, labels, text = hplot.get_histo(ResoPlot, ['ele-V9'], 'PU200', ['EG'], ['EGq5'], 'GENEtaBC', debug=False)

dm.addHistos([his.g_pt_resp_eff_sigma for his in hsets], ['endcap'])
# dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['endcap g'])


hsets, labels, text = hplot.get_histo(ResoPlot, ['ele-V9'], 'PU200', ['EGBRL'], ['LooseTkID'], 'GENEtaF', debug=False)
dm.addHistos([his.g_pt_resp_eff_sigma for his in hsets], ['barrel'])
# dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['barrel g'])

hsets, labels, text = hplot.get_histo(ResoPlot, ['ele-V9'], 'PU200', ['EG'], ['EGq5'], 'GENEtaD', debug=False)

dm.addHistos([his.g_pt_resp_eff_sigma for his in hsets], ['endcap++'])

dm.draw(text=text, options='', y_min=0., y_max=1.2)


In [ ]:
dm = DrawMachine(rleg_config)
hsets, labels, text = hplot.get_histo(ResoPlot, ['ele-V9'], 'PU200', ['EG'], ['EGq5'], 'GENEtaBC', debug=False)

dm.addHistos([his.g_pt_resp_cb_chi2prob for his in hsets], ['endcap'])
# dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['endcap g'])


hsets, labels, text = hplot.get_histo(ResoPlot, ['ele-V9'], 'PU200', ['EGBRL'], ['LooseTkID'], 'GENEtaF', debug=False)
dm.addHistos([his.g_pt_resp_cb_chi2prob for his in hsets], ['barrel'])
# dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['barrel g'])

hsets, labels, text = hplot.get_histo(ResoPlot, ['ele-V9'], 'PU200', ['EG'], ['EGq5'], 'GENEtaD', debug=False)

dm.addHistos([his.g_pt_resp_cb_chi2prob for his in hsets], ['endcap++'])

dm.draw(text=text, options='', y_min=0., y_max=1.2)


In [ ]:
dm = DrawMachine(rleg_config)
hsets, labels, text = hplot.get_histo(ResoPlot, ['ele-V9'], 'PU200', ['EG'], ['EGq5'], 'GENEtaBC', debug=False)

dm.addHistos([his.g_pt_resp_cb_sigma for his in hsets], ['endcap'])
# dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['endcap g'])


hsets, labels, text = hplot.get_histo(ResoPlot, ['ele-V9'], 'PU200', ['EGBRL'], ['LooseTkID'], 'GENEtaF', debug=False)
dm.addHistos([his.g_pt_resp_cb_sigma for his in hsets], ['barrel'])
# dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['barrel g'])

hsets, labels, text = hplot.get_histo(ResoPlot, ['ele-V9'], 'PU200', ['EG'], ['EGq5'], 'GENEtaDE', debug=False)

dm.addHistos([his.g_pt_resp_cb_sigma for his in hsets], ['endcap++'])

dm.draw(text=text, options='', y_min=0., y_max=0.3)


In [ ]:
dm = DrawMachine(rleg_config)
hsets, labels, text = hplot.get_histo(ResoPlot, ['ele-V9'], 'PU200', ['EG'], ['EGq5'], 'GENEtaBC', debug=False)

dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['endcap'])


hsets, labels, text = hplot.get_histo(ResoPlot, ['ele-V9'], 'PU200', ['EGBRL'], ['LooseTkID'], 'GENEtaF', debug=False)
dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['barrel'])

hsets, labels, text = hplot.get_histo(ResoPlot, ['ele-V9'], 'PU200', ['EG'], ['EGq5'], 'GENEtaD', debug=False)

dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['endcap++'])

dm.draw(text=text, options='', y_min=0., y_max=0.3)


In [ ]:
dm = DrawMachine(rleg_config)
hsets, labels, text = hplot.get_histo(ResoPlot, ['ele-V9'], 'PU200', ['EG'], ['EGq5'], 'GENEtaBC', debug=False)

dm.addHistos([his.g_pt_resp_gauss_avg for his in hsets], ['endcap'])


hsets, labels, text = hplot.get_histo(ResoPlot, ['ele-V9'], 'PU200', ['EGBRL'], ['LooseTkID'], 'GENEtaF', debug=False)
dm.addHistos([his.g_pt_resp_gauss_avg for his in hsets], ['barrel'])

hsets, labels, text = hplot.get_histo(ResoPlot, ['ele-V9'], 'PU200', ['EG'], ['EGq5'], 'GENEtaDE', debug=False)

dm.addHistos([his.g_pt_resp_gauss_avg for his in hsets], ['endcap++'])

dm.draw(text=text, options='', y_min=0.8, y_max=1.2)


In [ ]:
dm = DrawMachine(rleg_config)
hsets, labels, text = hplot.get_histo(ResoPlot, ['ele-V9'], 'PU200', ['EG'], ['EGq5'], 'GENEtaBC', debug=False)

dm.addHistos([his.g_pt_resp_cb_avg for his in hsets], ['endcap'])


hsets, labels, text = hplot.get_histo(ResoPlot, ['ele-V9'], 'PU200', ['EGBRL'], ['LooseTkID'], 'GENEtaF', debug=False)
dm.addHistos([his.g_pt_resp_cb_avg for his in hsets], ['barrel'])

hsets, labels, text = hplot.get_histo(ResoPlot, ['ele-V9'], 'PU200', ['EG'], ['EGq5'], 'GENEtaDE', debug=False)

dm.addHistos([his.g_pt_resp_cb_avg for his in hsets], ['endcap++'])

dm.draw(text=text, options='', y_min=0.8, y_max=1.2)


In [ ]:
dm = DrawMachine(rleg_config)
hsets, labels, text = hplot.get_histo(ResoPlot, ['photons-V9'], 'PU200', ['EG'], ['EGq5'], 'GENEtaBC', debug=False)

dm.addHistos([his.g_pt_resp_cb_sigma for his in hsets], ['endcap'])


hsets, labels, text = hplot.get_histo(ResoPlot, ['photons-V9'], 'PU200', ['EGBRL'], ['LooseTkID'], 'GENEtaF', debug=False)
dm.addHistos([his.g_pt_resp_cb_sigma for his in hsets], ['barrel'])

hsets, labels, text = hplot.get_histo(ResoPlot, ['photons-V9'], 'PU200', ['EG'], ['EGq5'], 'GENEtaDE', debug=False)

dm.addHistos([his.g_pt_resp_cb_sigma for his in hsets], ['endcap++'])

dm.draw(text=text, options='', y_min=0., y_max=0.6)


In [ ]:
dm = DrawMachine(rleg_config)
hsets, labels, text = hplot.get_histo(ResoPlot, ['ele-V9'], 'PU200', ['TkEleEL'], ['EGq5'], 'GENEtaBC', debug=False)

dm.addHistos([his.g_pt_resp_cb_chi2prob for his in hsets], ['endcap'])
# dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['endcap g'])


hsets, labels, text = hplot.get_histo(ResoPlot, ['ele-V9'], 'PU200', ['TkEleELBRL'], ['all'], 'GENEtaF', debug=False)
dm.addHistos([his.g_pt_resp_cb_chi2prob for his in hsets], ['barrel'])
# dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['barrel g'])


dm.draw(text=text, options='', y_min=0., y_max=1.2)


In [ ]:
print colors
print rleg_config.marker_styles.extend([33, 33, 33, 33, 33, 33, 33])

In [ ]:
dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(ResoPlot, ['ele-V9'], 'PU200', ['EG'], ['EGq5'], 'GENEtaBC', debug=False)
# dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['endcap'])
# dm.addHistos([his.g_pt_resp_cb_sigma for his in hsets], ['endcap CB'])
dm.addHistos([his.g_pt_resp_eff_sigma for his in hsets], ['endcap sta eff'])

hsets, labels, text = hplot.get_histo(ResoPlot, ['ele-V9'], 'PU200', ['TkEleEL'], ['EGq5'], 'GENEtaBC', debug=False)
# dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['endcap'])
# dm.addHistos([his.g_pt_resp_cb_sigma for his in hsets], ['endcap CB'])
dm.addHistos([his.g_pt_resp_eff_sigma for his in hsets], ['endcap eff'])

hsets, labels, text = hplot.get_histo(ResoPlot, ['ele-V9'], 'PU200', ['EGBRL'], ['LooseTkID'], 'GENEtaF', debug=False)
# dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['barrel'])
# dm.addHistos([his.g_pt_resp_cb_sigma for his in hsets], ['barrel CB'])
dm.addHistos([his.g_pt_resp_eff_sigma for his in hsets], ['barrel STA eff'])


hsets, labels, text = hplot.get_histo(ResoPlot, ['ele-V9'], 'PU200', ['TkEleELBRL'], ['all'], 'GENEtaF', debug=False)
# dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['barrel'])
# dm.addHistos([his.g_pt_resp_cb_sigma for his in hsets], ['barrel CB'])
dm.addHistos([his.g_pt_resp_eff_sigma for his in hsets], ['barrel eff'])

hsets, labels, text = hplot.get_histo(ResoPlot, ['ele-V9'], 'PU200', ['EG'], ['EGq5'], 'GENEtaD', debug=False)
dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['endcap ++'])
dm.addHistos([his.g_pt_resp_cb_sigma for his in hsets], ['endcap ++ CB'])
# dm.addHistos([his.g_pt_resp_eff_sigma for his in hsets], ['endcap ++ sta eff'])


dm.draw(text=text, options='', y_min=0., y_max=0.2)


In [ ]:
dm = DrawMachine(tdr_config)


# hsets, labels, text = hplot.get_histo(ResoPlot, ['ele-V9'], 'PU200', ['TkEleEL'], ['EGq5'], 'GENEtaBC', debug=False)
# # dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['endcap'])
# # dm.addHistos([his.g_pt_resp_cb_sigma for his in hsets], ['endcap CB'])
# dm.addHistos([his.g_pt_resp_eff_sigma for his in hsets], ['endcap eff'])

hsets, labels, text = hplot.get_histo(ResoPlot, ['ele-V9'], 'PU200', ['EGBRL'], ['LooseTkID'], 'GENEtaF', debug=False)
# dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['barrel'])
# dm.addHistos([his.g_pt_resp_cb_sigma for his in hsets], ['barrel CB'])
dm.addHistos([his.g_pt_resp_eff_sigma for his in hsets], ['|#eta| #leq 1.49'])

# hsets, labels, text = hplot.get_histo(ResoPlot, ['ele-V9'], 'PU200', ['EG'], ['EGq5'], 'GEN', debug=False)
# # dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['endcap'])
# # dm.addHistos([his.g_pt_resp_cb_sigma for his in hsets], ['endcap CB'])
# dm.addHistos([his.g_pt_resp_eff_sigma for his in hsets], ['1.52 < |#eta| #leq 2.4 all'])


hsets, labels, text = hplot.get_histo(ResoPlot, ['ele-V9'], 'PU200', ['EG'], ['EGq5'], 'GENEtaBC', debug=False)
# dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['endcap'])
# dm.addHistos([his.g_pt_resp_cb_sigma for his in hsets], ['endcap CB'])
dm.addHistos([his.g_pt_resp_eff_sigma for his in hsets], ['1.52 < |#eta| #leq 2.4'])

# hsets, labels, text = hplot.get_histo(ResoPlot, ['ele-V9'], 'PU200', ['TkEleELBRL'], ['all'], 'GENEtaF', debug=False)
# # dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['barrel'])
# # dm.addHistos([his.g_pt_resp_cb_sigma for his in hsets], ['barrel CB'])
# dm.addHistos([his.g_pt_resp_eff_sigma for his in hsets], ['barrel eff'])

hsets, labels, text = hplot.get_histo(ResoPlot, ['ele-V9'], 'PU200', ['EG'], ['EGq5'], 'GENEtaD', debug=False)
# dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['endcap ++'])
# dm.addHistos([his.g_pt_resp_cb_sigma for his in hsets], ['endcap ++ CB'])
dm.addHistos([his.g_pt_resp_eff_sigma for his in hsets], ['2.4 < |#eta| #leq 2.8'])

# hsets, labels, text = hplot.get_histo(ResoPlot, ['ele-V9'], 'PU200', ['EG'], ['EGq5'], 'GENEtaDE', debug=False)
# # dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['endcap ++'])
# # dm.addHistos([his.g_pt_resp_cb_sigma for his in hsets], ['endcap ++ CB'])
# dm.addHistos([his.g_pt_resp_eff_sigma for his in hsets], ['2.4 < |#eta| #leq 3.0'])


dm.draw(text='EG, calorimeter-only', options='', y_min=0., y_max=0.35, 
        y_axis_label='#sigma^{EFF(68%)}(p_{T}^{L1}/p_{T}^{GEN})', 
        x_axis_label='p_{T}^{GEN} [GeV]')


In [ ]:
dm = DrawMachine(tdr_config)


# # dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['endcap'])
# # dm.addHistos([his.g_pt_resp_cb_sigma for his in hsets], ['endcap CB'])
# dm.addHistos([his.g_pt_resp_eff_sigma for his in hsets], ['endcap eff'])
hsets, labels, text = hplot.get_histo(ResoPlot, ['ele-V9'], 'PU200', ['TkEleELBRL'], ['all'], 'GENEtaF', debug=False)

# hsets, labels, text = hplot.get_histo(ResoPlot, ['ele-V9'], 'PU200', ['EGBRL'], ['LooseTkID'], 'GENEtaF', debug=False)
# dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['barrel'])
# dm.addHistos([his.g_pt_resp_cb_sigma for his in hsets], ['barrel CB'])
dm.addHistos([his.g_pt_resp_eff_sigma for his in hsets], ['|#eta| #leq 1.479'])


hsets, labels, text = hplot.get_histo(ResoPlot, ['ele-V9'], 'PU200', ['TkEleEL'], ['EGq5'], 'GENEtaBC', debug=False)

# hsets, labels, text = hplot.get_histo(ResoPlot, ['ele-V9'], 'PU200', ['EG'], ['EGq5'], 'GENEtaBC', debug=False)
# dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['endcap'])
# dm.addHistos([his.g_pt_resp_cb_sigma for his in hsets], ['endcap CB'])
dm.addHistos([his.g_pt_resp_eff_sigma for his in hsets], ['1.52 #leq |#eta| #leq 2.4'])

# # dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['barrel'])
# # dm.addHistos([his.g_pt_resp_cb_sigma for his in hsets], ['barrel CB'])
# dm.addHistos([his.g_pt_resp_eff_sigma for his in hsets], ['barrel eff'])



dm.draw(text='EG, track-matched', options='', y_min=0., y_max=0.15, 
        y_axis_label='RMS^{Eff.(68%)}(p_{T}^{L1}/p_{T}^{GEN})', 
        x_axis_label='p_{T}^{GEN} [GeV]')

dm.write('figs/egamma_TkEle_response_effrms')


In [ ]:
dm = DrawMachine(tdr_config)


# # dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['endcap'])
# # dm.addHistos([his.g_pt_resp_cb_sigma for his in hsets], ['endcap CB'])
# dm.addHistos([his.g_pt_resp_eff_sigma for his in hsets], ['endcap eff'])
hsets, labels, text = hplot.get_histo(ResoPlot, ['ele-V9'], 'PU200', ['TkEleELBRL'], ['all'], 'GENEtaF', debug=False)

# hsets, labels, text = hplot.get_histo(ResoPlot, ['ele-V9'], 'PU200', ['EGBRL'], ['LooseTkID'], 'GENEtaF', debug=False)
# dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['barrel'])
# dm.addHistos([his.g_pt_resp_cb_sigma for his in hsets], ['barrel CB'])
dm.addHistos([his.g_pt_resp_gauss_avg for his in hsets], ['|#eta| #leq 1.49'])
dm.addHistos([his.g_pt_resp_cb_avg for his in hsets], ['|#eta| #leq 1.49 CB'])


hsets, labels, text = hplot.get_histo(ResoPlot, ['ele-V9'], 'PU200', ['TkEleEL'], ['EGq5'], 'GENEtaBC', debug=False)

# hsets, labels, text = hplot.get_histo(ResoPlot, ['ele-V9'], 'PU200', ['EG'], ['EGq5'], 'GENEtaBC', debug=False)
# dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['endcap'])
# dm.addHistos([his.g_pt_resp_cb_sigma for his in hsets], ['endcap CB'])
dm.addHistos([his.g_pt_resp_gauss_avg for his in hsets], ['1.52 #leq |#eta| #leq 2.4'])
dm.addHistos([his.g_pt_resp_cb_avg for his in hsets], ['1.52 #leq |#eta| #leq 2.4 CB'])

# # dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['barrel'])
# # dm.addHistos([his.g_pt_resp_cb_sigma for his in hsets], ['barrel CB'])
# dm.addHistos([his.g_pt_resp_eff_sigma for his in hsets], ['barrel eff'])



dm.draw(text='EG, track-matched', options='', y_min=0.8, y_max=1.2, 
        y_axis_label='<p_{T}^{L1}/p_{T}^{GEN}>', 
        x_axis_label='p_{T}^{GEN} [GeV]')



In [ ]:
dm = DrawMachine(tdr_config)


# # dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['endcap'])
# # dm.addHistos([his.g_pt_resp_cb_sigma for his in hsets], ['endcap CB'])
# dm.addHistos([his.g_pt_resp_eff_sigma for his in hsets], ['endcap eff'])
hsets, labels, text = hplot.get_histo(ResoPlot, ['ele-V9'], 'PU200', ['TkEleELBRL'], ['all'], 'GENEtaF', debug=False)

# hsets, labels, text = hplot.get_histo(ResoPlot, ['ele-V9'], 'PU200', ['EGBRL'], ['LooseTkID'], 'GENEtaF', debug=False)
# dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['barrel'])
# dm.addHistos([his.g_pt_resp_cb_sigma for his in hsets], ['barrel CB'])
dm.addHistos([his.g_pt_resp_rms for his in hsets], ['|#eta| #leq 1.49'])


hsets, labels, text = hplot.get_histo(ResoPlot, ['ele-V9'], 'PU200', ['TkEleEL'], ['EGq5'], 'GENEtaBC', debug=False)

# hsets, labels, text = hplot.get_histo(ResoPlot, ['ele-V9'], 'PU200', ['EG'], ['EGq5'], 'GENEtaBC', debug=False)
# dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['endcap'])
# dm.addHistos([his.g_pt_resp_cb_sigma for his in hsets], ['endcap CB'])
dm.addHistos([his.g_pt_resp_rms for his in hsets], ['1.52 < |#eta| #leq 2.4'])

# # dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['barrel'])
# # dm.addHistos([his.g_pt_resp_cb_sigma for his in hsets], ['barrel CB'])
# dm.addHistos([his.g_pt_resp_eff_sigma for his in hsets], ['barrel eff'])



dm.draw(text='EG, track-matched', options='', y_min=0., y_max=0.15, 
        y_axis_label='#sigma^{EFF(68%)}(p_{T}^{L1}/p_{T}^{GEN})', 
        x_axis_label='p_{T}^{GEN} [GeV]')


### Compare different EG qualities

In [ ]:
dm = DrawMachine(rleg_config)
hsets, labels, text = hplot.get_histo(ResoPlot, 'photons-V9', 'PU0', ['EG'], ['EGq5', 'EGq4', 'EGq3'], 'GENEtaBC', debug=False)

# dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['gauss.'])
# dm.addHistos([his.g_pt_resp_cb_sigma for his in hsets], ['CB'])
dm.addHistos([his.g_pt_resp_eff_sigma for his in hsets], labels)
# dm.addHistos([his.g_pt_resp_rms for his in hsets], ['rms'])

# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew for his in hsets], ['new'])
# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew2 for his in hsets], ['effSigma'])

dm.draw(text=text, options='COLZ', y_min=0., y_max=0.1)



In [ ]:
dm = DrawMachine(rleg_config)
hsets, labels, text = hplot.get_histo(ResoPlot, 'photons-V9', 'PU0', ['EG'], ['EGq5', 'EGq4', 'EGq3'], 'GENEtaBC', debug=False)

# dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['gauss.'])
# dm.addHistos([his.g_pt_resp_cb_sigma for his in hsets], ['CB'])
dm.addHistos([his.g_pt_resp_rms for his in hsets], labels)
# dm.addHistos([his.g_pt_resp_rms for his in hsets], ['rms'])

# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew for his in hsets], ['new'])
# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew2 for his in hsets], ['effSigma'])

dm.draw(text=text, options='COLZ', y_min=0., y_max=0.1)


In [ ]:
dm = DrawMachine(rleg_config)
hsets, labels, text = hplot.get_histo(ResoPlot, 'ele-V9', 'PU0', ['EG'], ['EGq5', 'EGq4', 'EGq3'], 'GENEtaBC', debug=False)

# dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['gauss.'])
# dm.addHistos([his.g_pt_resp_cb_sigma for his in hsets], ['CB'])
dm.addHistos([his.g_pt_resp_eff_sigma for his in hsets], labels)
# dm.addHistos([his.g_pt_resp_rms for his in hsets], ['rms'])

# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew for his in hsets], ['new'])
# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew2 for his in hsets], ['effSigma'])

dm.draw(text=text, options='COLZ', y_min=0., y_max=0.1)



In [ ]:
dm = DrawMachine(rleg_config)
hsets, labels, text = hplot.get_histo(ResoPlot, 'ele-V9', 'PU0', ['EG'], ['EGq5', 'EGq4', 'EGq3'], 'GENEtaBC', debug=False)

# dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['gauss.'])
# dm.addHistos([his.g_pt_resp_cb_sigma for his in hsets], ['CB'])
dm.addHistos([his.g_pt_resp_rms for his in hsets], labels)
# dm.addHistos([his.g_pt_resp_rms for his in hsets], ['rms'])

# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew for his in hsets], ['new'])
# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew2 for his in hsets], ['effSigma'])

dm.draw(text=text, options='COLZ', y_min=0., y_max=0.4)



In [ ]:
dm = DrawMachine(rleg_config)
hsets, labels, text = hplot.get_histo(ResoPlot, 'ele-V9', 'PU200', ['EG'], ['EGq5', 'EGq4', 'EGq3'], 'GENEtaBC', debug=False)

# dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['gauss.'])
# dm.addHistos([his.g_pt_resp_cb_sigma for his in hsets], ['CB'])
dm.addHistos([his.g_pt_resp_eff_sigma for his in hsets], labels)
# dm.addHistos([his.g_pt_resp_rms for his in hsets], ['rms'])

# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew for his in hsets], ['new'])
# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew2 for his in hsets], ['effSigma'])

dm.draw(text=text, options='COLZ', y_min=0., y_max=0.4)



In [ ]:
dm = DrawMachine(rleg_config)
hsets, labels, text = hplot.get_histo(ResoPlot, 'ele-V9', 'PU200', ['EG'], ['EGq5', 'EGq4', 'EGq3'], 'GENEtaBC', debug=False)

# dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['gauss.'])
# dm.addHistos([his.g_pt_resp_cb_sigma for his in hsets], ['CB'])
dm.addHistos([his.g_pt_resp_rms for his in hsets], labels)
# dm.addHistos([his.g_pt_resp_rms for his in hsets], ['rms'])

# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew for his in hsets], ['new'])
# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew2 for his in hsets], ['effSigma'])

dm.draw(text=text, options='COLZ', y_min=0., y_max=0.4)



In [17]:
dm = DrawMachine(rleg_config)
hsets, labels, text = hplot.get_histo(ResoPlot, 'ele-V9', 'PU200', ['EG', 'HMvDREmInt', 'HMvDRshapeDrCalib', 'HMvDRshapeDrCalibNew'], ['EGq5', "Em"], 'GENEtaBC', debug=False)

# dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['gauss.'])
# dm.addHistos([his.g_pt_resp_cb_sigma for his in hsets], ['CB'])
dm.addHistos([his.g_pt_resp_cb_sigma for his in hsets], labels)
# dm.addHistos([his.g_pt_resp_rms for his in hsets], ['rms'])

# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew for his in hsets], ['new'])
# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew2 for his in hsets], ['effSigma'])

dm.draw(text=text, options='COLZ', y_min=0., y_max=0.4)



In [18]:
dm = DrawMachine(rleg_config)
hsets, labels, text = hplot.get_histo(ResoPlot, 'ele-V9', 'PU200', ['EG', 'HMvDREmInt', 'HMvDRshapeDrCalib', 'HMvDRshapeDrCalibNew'], ['EGq5', "Em"], 'GENEtaBC', debug=False)

# dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['gauss.'])
# dm.addHistos([his.g_pt_resp_cb_sigma for his in hsets], ['CB'])
dm.addHistos([his.g_pt_resp_cb_avg for his in hsets], labels)
# dm.addHistos([his.g_pt_resp_rms for his in hsets], ['rms'])

# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew for his in hsets], ['new'])
# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew2 for his in hsets], ['effSigma'])

dm.draw(text=text, options='COLZ', y_min=0., y_max=2)



In [19]:
dm = DrawMachine(rleg_config)
hsets, labels, text = hplot.get_histo(ResoPlot, 'ele-V9', 'PU200', ['EG', 'HMvDREmInt'], ['EGq5', "Em"], 'GENEtaBC', debug=False)

# dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['gauss.'])
# dm.addHistos([his.g_pt_resp_cb_sigma for his in hsets], ['CB'])
dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], labels=['before fix', 'after fix'])
# dm.addHistos([his.g_pt_resp_rms for his in hsets], ['rms'])

# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew for his in hsets], ['new'])
# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew2 for his in hsets], ['effSigma'])

dm.draw(text=text, options='COLZ', h_lines=[1.], y_min=0., y_max=0.4, y_axis_label='#sigma_{gauss}(p_{T}^{L1}/p_{T}^{GEN})', x_axis_label='p_{T}^{GEN} [GeV]')



In [20]:
dm = DrawMachine(rleg_config)
hsets, labels, text = hplot.get_histo(ResoPlot, 'ele-V9', 'PU200', ['EG', 'HMvDREmInt'], ['EGq5', "Em"], 'GENEtaBC', debug=False)

# dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['gauss.'])
# dm.addHistos([his.g_pt_resp_cb_sigma for his in hsets], ['CB'])
dm.addHistos([his.g_pt_resp_cb_avg for his in hsets], labels=['before fix', 'after fix'])
# dm.addHistos([his.g_pt_resp_rms for his in hsets], ['rms'])

# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew for his in hsets], ['new'])
# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew2 for his in hsets], ['effSigma'])

dm.draw(text=text, options='COLZ', h_lines=[1.], y_min=0., y_max=2, y_axis_label='<p_{T}^{L1}/p_{T}^{GEN}>', x_axis_label='p_{T}^{GEN} [GeV]')



In [21]:
dm = DrawMachine(rleg_config)
hsets, labels, text = hplot.get_histo(ResoPlot, 'ele-V9', 'PU200', ['EG', 'HMvDREmInt', 'HMvDRshapeDrCalibNew'], ['EGq5', "Em"], 'GENEtaBC', debug=False)

# dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['gauss.'])
# dm.addHistos([his.g_pt_resp_cb_sigma for his in hsets], ['CB'])
dm.addHistos([his.h_ptRespVeta for his in hsets], labels=['before fix', 'after fix', 'new calib'])
# dm.addHistos([his.g_pt_resp_rms for his in hsets], ['rms'])

# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew for his in hsets], ['new'])
# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew2 for his in hsets], ['effSigma'])

dm.draw(text=text, options='COLZ', h_lines=[1.], 
        y_min=0., y_max=2, 
        y_axis_label='<p_{T}^{L1}/p_{T}^{GEN}>', x_axis_label='|#eta|^{GEN}', do_profile=True)



In [23]:
dm = DrawMachine(rleg_config)
hsets, labels, text = hplot.get_histo(ResoPlot, 'ele-V9', 'PU200', ['EG', 'HMvDREmInt'], ['EGq5', "Em"], 'GENEtaBC', debug=False)

# dm.addHistos([his.g_pt_resp_gauss_sigma for his in hsets], ['gauss.'])
# dm.addHistos([his.g_pt_resp_cb_sigma for his in hsets], ['CB'])
dm.addHistos([his.pt_resp_binned[7] for his in hsets], labels)
# dm.addHistos([his.g_pt_resp_rms for his in hsets], ['rms'])

# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew for his in hsets], ['new'])
# dm.addHistos([his.g_energyResVenergy_raw_sigmaEOENew2 for his in hsets], ['effSigma'])

dm.draw(text=text, options='hist')



In [ ]:
import math
# z = 3190

x, y , z = 50.299686,  -3.625200,  319.814972


# R

eta = 1.5
delta_ro = 0.015

theta = 2*math.atan(math.exp(-1*eta))
theta_deg = 180*theta/math.pi
print 'eta: {} -> theta: {} = {} deg'.format(eta, theta, theta_deg)

r = z*math.tan(theta)
delta_r = delta_ro*z
theta_plus = math.atan2(r+delta_r, z)
theta_minus = math.atan2(r-delta_r, z)
eta_plus = -1*math.log(math.tan(theta_plus/2))
eta_minus = -1*math.log(math.tan(theta_minus/2))

delta_eta_plus = eta - eta_plus
delta_eta_minus = eta_minus - eta

print '     delta_eta_plus: {}, delta_eta_minus: {}'.format(delta_eta_plus, delta_eta_minus)

delta_phi = math.atan2(delta_r, r)

print '     delta_phi: {}'.format(delta_phi)

# x_proj = 



# components = tcs[tcs.id.isin(cluster.clusters)].copy()
#         components['R_cl'] = components.z/cluster.sinh_eta
#         components['x_cl'] = components.R_cl*cluster.cos_phi
#         components['y_cl'] = components.R_cl*cluster.sin_phi
#         components['dist2'] = (components.x_cl-components.x)**2+(components.y_cl-components.y)**2
#         components['dist'] = np.sqrt(components.dist2)
#         components['dr'] = components.dist/np.abs(components.z)
